## 목차
1. 데이터 준비  
    [1-1. 라이브러리 불러오기](#라이브러리-불러오기)  
    [1-2. 데이터 불러오기](#데이터-불러오기)  
    [1-3. 데이터 전처리](#데이터-전처리)
2. Prompt 작성
    [2-1. API키 받아오기](#api키-설정)
    [2-2. Prompt](#prompt-작성)
3. 개인 작업 및 임시 코드 작성
    [.](#여기서부터는-무시)    

# 라이브러리 불러오기

In [2]:
import pickle
import pandas as pd
import os
import openai
import json
from pydantic import BaseModel
import os
import pandas as pd
from jsonschema import validate, ValidationError
import openai

# 데이터 불러오기

In [3]:
# 피클 파일 경로
file_path = 'data/filter_df.pickle'

# 피클 파일 불러오기
with open(file_path, 'rb') as file:
    df = pickle.load(file)

# 데이터프레임 내용 출력
# df.head()

# 데이터 프레임으로 변환
df = pd.DataFrame(df)

# 데이터 전처리

In [4]:
# 불필요한 상품 삭제
df = df[~df['title'].str.contains('임대|야놀자|입장권|상품권|포인트|야놀|주차권|구매', na=False)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503 entries, 0 to 554
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   platform           503 non-null    object        
 1   original_link      503 non-null    object        
 2   post_time          503 non-null    datetime64[ns]
 3   title              503 non-null    object        
 4   view_count         503 non-null    int32         
 5   like_count         503 non-null    int32         
 6   price              503 non-null    int32         
 7   images             499 non-null    object        
 8   description        503 non-null    object        
 9   category           269 non-null    object        
 10  status             269 non-null    object        
 11  seller_location    175 non-null    object        
 12  title_nouns        503 non-null    object        
 13  seller_location_1  175 non-null    object        
dtypes: datetim

In [136]:
# 일단 pass 크롤링단계에서 확인 후 중복 제거할 예정. df.drop_duplicates(subset='title').info()
# 최신 or 처음 --> 

In [104]:
# 평균적으로 상세설명은 265글자 정도
sum(df['description'].apply(len)) / len(df['description'])

265.76739562624255

In [106]:
# 제거할 특정 문구 목록
specific_texts = [
    "※ 카페 상품 게시글은 자동으로 중고나라 앱/사이트에 노출합니다. 노출을 원하지 않으실 경우 고객센터로 문의 바랍니다.",
    "※ 등록한 게시글이 회원의 신고를 받거나 이상거래로 모니터링 될 경우 중고나라 사기통합조회 DB로 수집/활용될 수 있습니다.",
    "───────────────────",
    "📢 제목에 \"제조사/ 브랜드 명\"과 \"티켓명(ex. 스타벅스 아메리카노 Tall) ”을 넣어 작성하면, 보다 빠른 판매가 가능합니다!",
    "📢 게시글 작성 시 배송 방법에 “직거래”와 “내 위치” 설정할 경우, 보다 빠른 판매가 가능합니다!"
]

# 특정 문구 제거 함수
def remove_specific_texts(description, specific_texts):
    for text in specific_texts:
        description = description.replace(text, "")
    return description

# description 열에서 특정 문구 제거
df['cleaned_description'] = df['description'].apply(lambda x: remove_specific_texts(x, specific_texts))

# 각 description의 길이 측정
description_lengths = df[['description', 'cleaned_description']].assign(
    original_length=df['description'].str.len(),
    cleaned_length=df['cleaned_description'].str.len()
)

# cleaned_length가 20 이하인 인덱스를 추출
filtered_indices = description_lengths[description_lengths['cleaned_length'] <= 15].index

# 원본 데이터프레임에서 해당 인덱스의 행을 가져옴
filtered_df = df.loc[filtered_indices]

# 결과 출력
filtered_df

,platform,original_link,post_time,title,view_count,like_count,price,images,description,category,status,seller_location,title_nouns,seller_location_1,cleaned_description
3,당근마켓,https://www.daangn.com/articles/840986215,2024-10-02 12:35:31.443000,파타야 힐튼호텔 디럭스룸 10/12~13,21,0,400000,https://img.kr.gcp-karroter.net/origin/article...,연락주세요,티켓/교환권,판매중,수원시 영통구 영통동,"[파타야, 힐튼, 호텔, 디럭스, 룸]",수원시,연락주세요
46,당근마켓,https://www.daangn.com/articles/840864099,2024-10-02 08:54:31.444000,수성호텔 하와이 115000원 이용권 인피니티 루프탑 사우나 온천 종합 이용권,68,0,45000,https://img.kr.gcp-karroter.net/origin/article...,1장 남았어요,티켓/교환권,판매중,북구 침산동,"[수성, 호텔, 하와이, 이용권, 인피니티, 루프, 탑, 사우나, 온천, 종합, 이용권]",북구,1장 남았어요
102,번개장터,https://bunjang.co.kr/products/291752672,2024-10-02 01:26:02.327000,신라스테이 삼성코엑스 호텔 10.6-7 1박 양도,20,1,140000,https://media.bunjang.co.kr/product/291752672_...,트원룸 \n2인조식 포함,여행/숙박/렌트,판매중,NaN,"[신라, 스테이, 삼성, 코엑스, 호텔, 박, 양도]",NaN,트원룸 \n2인조식 포함
134,번개장터,https://bunjang.co.kr/products/281522481,2024-09-30 13:26:02.327000,신라호텔 숙박권,172,2,1000000,https://media.bunjang.co.kr/product/281522481_...,이그제큐티브 그랜드 디럭스룸,여행/숙박/렌트,판매중,NaN,"[신라, 호텔, 숙박]",NaN,이그제큐티브 그랜드 디럭스룸
369,중고나라,https://www.중고나라.com/items/,2024-10-01 20:26:53.006777,웰리힐리 리조트 패밀리형 88m2 10/4(금),3,0,160000,https://img2.joongna.com/cafe-article-data/liv...,※ 카페 상품 게시글은 자동으로 중고나라 앱/사이트에 노출합니다. 노출을 원하지 않...,NaN,NaN,NaN,"[웰리, 힐리, 리조트, 패밀리, 금]",NaN,\n \n \n \n
499,번개장터,https://bunjang.co.kr/products/281522481,2024-09-30 16:38:37.342000,신라호텔 숙박권,177,3,1000000,https://media.bunjang.co.kr/product/281522481_...,이그제큐티브 그랜드 디럭스룸,여행/숙박/렌트,판매중,NaN,"[신라, 호텔, 숙박]",NaN,이그제큐티브 그랜드 디럭스룸
512,번개장터,https://bunjang.co.kr/products/234208202,2024-10-04 16:38:37.342000,호텔 양도,22,1,148000,https://media.bunjang.co.kr/product/234208202_...,🧡🩶🧡🩶🧡🩶🧡🩶,여행/숙박/렌트,판매완료,서북구 부성1동,"[호텔, 양도]",서북구,🧡🩶🧡🩶🧡🩶🧡🩶


In [ ]:
# cleaned_length가 20 이하인 인덱스를 추출
filtered_indices = description_lengths[description_lengths['cleaned_length'] >= 700].index

# 원본 데이터프레임에서 해당 인덱스의 행을 가져옴
filtered_df = df.loc[filtered_indices]

# 결과 출력
filtered_df

In [131]:
# 평균적으로 상세설명은 265글자 정도
sum(filtered_df['cleaned_description'].apply(len)) / len(filtered_df['cleaned_description'])

1044.9333333333334

# API키 설정

In [5]:
# OpenAI API Key를 환경 변수에서 가져오기
openai.api_key = os.getenv("OPEN_API_KEY")

# prompt 작성

In [7]:
import concurrent.futures
import openai
import json

# JSON 스키마 정의
hotel_schema = {
    "type": "object",
    "properties": {
        "expiration_start_date": {"type": ["string", "null"], "format": "date"},
        "expiration_end_date": {"type": ["string", "null"], "format": "date"},
        "room_type": {"type": ["string", "null"]},
        "head_count": {"type": ["integer", "null"]},
        "shipping_fee": {"type": "integer"},
        "transaction_method": {"type": ["string", "null"]},
        "market_price": {"type": ["integer", "null"]},
        "options": {"type": ["string", "null"]},
        "parking": {"type": ["boolean", "null"]},
        "check_in_time": {"type": ["string", "null"], "format": "time"},
        "check_out_time": {"type": ["string", "null"], "format": "time"},
        "sub_category": {"type": ["string", "null"]}
    },
    "required": ["expiration_start_date", "expiration_end_date", "room_type", "head_count", "shipping_fee", "transaction_method", "market_price", "options", "parking", "check_in_time", "check_out_time", "sub_category"]
}

# ChatGPT API를 통해 정보를 추출하는 함수
def extract_hotel_data(rows):
    extracted_data_list = []
    
    # 각 행에 대해 반복
    for index, row in rows.iterrows():
        title = row['title']
        description = row['description']
        post_time = row['post_time']
        
        # 채팅 기반 프롬프트 생성
        prompt = f"""
        제목: {title}
        설명: {description}
        업로드 시간: {post_time}

        위 정보를 바탕으로 노쇼 상품에 대한 다음 정보를 추출해주세요:
        
        1. 사용 가능 시작 날짜 (expiration_start_date): 시작 날짜를 추출해주세요. 만약 정확한 날짜가 없이 시간 대명사로 되어 있다면 업로드 시간을 참조하여 계산하여 표기해주세요. 날짜 형식은 '20XX-MM-DD'입니다. 시작 날짜가 없으면 'null'로 표기해주세요.
        2. 사용 가능 마지막 날짜 (expiration_end_date): 종료 날짜를 추출해주세요. 만약 정확한 날짜가 없이 시간 대명사로 되어 있다면 업로드 시간을 참조하여 계산하여 표기해주세요. 날짜 형식은 '20XX-MM-DD'입니다. 마지막 날짜가 없으면 'null'로 표기해주세요.
        3. 객실 타입 (room_type): 호텔 객실 타입을 추출해주세요. 관련된 정보가 없다면 'null'로 표기해주세요.
        4. 사용 인원 수 (head_count): 설명에 언급된 인원 수를 추출해주세요. 인원 수가 없다면 'null'로 표기해주세요.
        5. 배송비 (shipping_fee): 설명에 배송비가 있으면 숫자로 표기해주세요. 없으면 0으로 표기해주세요.
        6. 거래 유형 (transaction_method): 직거래나 택배거래 정보를 추출해주세요. 없으면 'null'로 표기해주세요.
        7. 시중 가격 (market_price): 원래 상품의 가격을 추출해주세요. 가격이 없으면 'null'로 표기해주세요. 숫자 형식으로 표기해주세요. 예: 233000.
        8. 서비스 옵션 (options): 조식, 사우나, 수영장 등의 서비스 옵션이 있는지 확인하고 적어주세요. 없으면 'null'로 표기해주세요.
        9. 주차 여부 (parking): 주차 가능 여부를 확인하고 TRUE 또는 FALSE로 표기해주세요. 없으면 'null'로 표기해주세요.
        10. 체크인 시간 (check_in_time): 체크인 시간을 추출해주세요. 없으면 'null'로 표기해주세요. 시간 형식은 'HH:mm'입니다.
        11. 체크아웃 시간 (check_out_time): 체크아웃 시간을 추출해주세요. 없으면 'null'로 표기해주세요. 시간 형식은 'HH:mm'입니다.
        12. 세부 카테고리 (sub_category): 제목과 설명을 참조하여 호텔,리조트,모텔,펜션,콘도,풀빌라,글램핕,캠핑 등의 카테고리를 추출해주세요. 관련된 정보가 없다면 'null'로 표기해주세요.
        """
        
        # ChatGPT Chat API 호출, 구조적 출력 요청
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": prompt}],
            max_tokens=300,
            temperature=0.5
        )
        
        # 응답에서 메시지 내용 추출
        response_content = response['choices'][0]['message']['content']
        
        # 응답 내용에서 필드 추출 (예시: 각 항목을 별도로 분리하는 로직 필요)
        extracted_data = {
            "expiration_start_date": None,
            "expiration_end_date": None,
            "room_type": None,
            "head_count": None,
            "shipping_fee": 0,
            "transaction_method": None,
            "market_price": None,
            "options": None,
            "parking": None,
            "check_in_time": None,
            "check_out_time": None,
            "sub_category": None
        }
        
        # 응답 내용에서 필드를 채우는 방법을 개발해야 함
        # 여기서는 각 항목을 수동으로 처리하거나 특정 패턴을 따르는 문자열 분리로 필드를 추출하는 로직이 필요합니다.
        # 예시로 'expiration_start_date'를 추출하는 로직을 추가하세요.

        # 예시 추출 (임시로 응답을 직접 할당하는 방식)
        extracted_data["expiration_start_date"] = "2024-10-05"
        extracted_data["expiration_end_date"] = "2024-10-06"
        extracted_data["room_type"] = "특급 스위트"
        extracted_data["head_count"] = 2
        extracted_data["shipping_fee"] = 5000
        extracted_data["transaction_method"] = "직거래"
        extracted_data["market_price"] = 50000
        extracted_data["options"] = "조식포함, 무료와이파이"
        extracted_data["parking"] = True
        extracted_data["check_in_time"] = "15:00"
        extracted_data["check_out_time"] = "11:00"
        extracted_data["sub_category"] = "호텔"

        extracted_data_list.append(extracted_data)

    return extracted_data_list

# 데이터프레임의 각 행에 대해 데이터 추출 및 검증
for index, row in df.iterrows():
    extracted_data = extract_hotel_data(row)
    
    # JSON 형식으로 출력
    print(json.dumps(extracted_data, indent=4, ensure_ascii=False))
    print(extracted_data)

AttributeError: 'Series' object has no attribute 'iterrows'

In [19]:
test_df = df.iloc[:10,:]
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   platform           10 non-null     object        
 1   original_link      10 non-null     object        
 2   post_time          10 non-null     datetime64[ns]
 3   title              10 non-null     object        
 4   view_count         10 non-null     int32         
 5   like_count         10 non-null     int32         
 6   price              10 non-null     int32         
 7   images             10 non-null     object        
 8   description        10 non-null     object        
 9   category           10 non-null     object        
 10  status             10 non-null     object        
 11  seller_location    10 non-null     object        
 12  title_nouns        10 non-null     object        
 13  seller_location_1  10 non-null     object        
dtypes: datetime64

In [45]:
import openai
import json
import concurrent.futures

# JSON 스키마 정의
hotel_schema = {
    "type": "object",
    "properties": {
        "expiration_start_date": {"type": ["string", "null"], "format": "date"},
        "expiration_end_date": {"type": ["string", "null"], "format": "date"},
        "room_type": {"type": ["string", "null"]},
        "head_count": {"type": ["integer", "null"]},
        "shipping_fee": {"type": "integer"},
        "transaction_method": {"type": ["string", "null"]},
        "market_price": {"type": ["integer", "null"]},
        "options": {"type": ["string", "null"]},
        "parking": {"type": ["boolean", "null"]},
        "check_in_time": {"type": ["string", "null"], "format": "time"},
        "check_out_time": {"type": ["string", "null"], "format": "time"},
        "sub_category": {"type": ["string", "null"]}
    },
    "required": ["expiration_start_date", "expiration_end_date", "room_type", "head_count", "shipping_fee", "transaction_method", "market_price", "options", "parking", "check_in_time", "check_out_time", "sub_category"]
}

# ChatGPT API를 통해 정보를 추출하는 함수
def extract_hotel_data(rows):
    extracted_data_list = []
    
    # 각 행에 대해 반복
    for index, row in rows.iterrows():
        title = row['title']
        description = row['description']
        post_time = row['post_time']
        
        # 채팅 기반 프롬프트 생성
        prompt = f"""
        제목: {title}
        설명: {description}
        업로드 시간: {post_time}

        위 정보를 바탕으로 노쇼 상품에 대한 다음 정보를 추출해주세요:
        
        1. 사용 가능 시작 날짜 (expiration_start_date): 시작 날짜를 추출해주세요. 만약 정확한 날짜가 없이 시간 대명사로 되어 있다면 업로드 시간을 참조하여 계산하여 표기해주세요. 날짜 형식은 '20XX-MM-DD'입니다. 시작 날짜가 없으면 'null'로 표기해주세요.
        2. 사용 가능 마지막 날짜 (expiration_end_date): 종료 날짜를 추출해주세요. 만약 정확한 날짜가 없이 시간 대명사로 되어 있다면 업로드 시간을 참조하여 계산하여 표기해주세요. 날짜 형식은 '20XX-MM-DD'입니다. 마지막 날짜가 없으면 'null'로 표기해주세요.
        3. 객실 타입 (room_type): 호텔 객실 타입을 추출해주세요. 관련된 정보가 없다면 'null'로 표기해주세요.
        4. 사용 인원 수 (head_count): 설명에 언급된 인원 수를 추출해주세요. 인원 수가 없다면 'null'로 표기해주세요.
        5. 배송비 (shipping_fee): 설명에 배송비가 있으면 숫자로 표기해주세요. 없으면 0으로 표기해주세요.
        6. 거래 유형 (transaction_method): 직거래나 택배거래 정보를 추출해주세요. 없으면 'null'로 표기해주세요.
        7. 시중 가격 (market_price): 원래 상품의 가격을 추출해주세요. 가격이 없으면 'null'로 표기해주세요. 숫자 형식으로 표기해주세요. 예: 233000.
        8. 서비스 옵션 (options): 조식, 사우나, 수영장 등의 서비스 옵션이 있는지 확인하고 적어주세요. 없으면 'null'로 표기해주세요.
        9. 주차 여부 (parking): 주차 가능 여부를 확인하고 TRUE 또는 FALSE로 표기해주세요. 없으면 'null'로 표기해주세요.
        10. 체크인 시간 (check_in_time): 체크인 시간을 추출해주세요. 없으면 'null'로 표기해주세요. 시간 형식은 'HH:mm'입니다.
        11. 체크아웃 시간 (check_out_time): 체크아웃 시간을 추출해주세요. 없으면 'null'로 표기해주세요. 시간 형식은 'HH:mm'입니다.
        12. 세부 카테고리 (sub_category): 제목과 설명을 참조하여 호텔,리조트,모텔,펜션,콘도,풀빌라,글램핕,캠핑 등의 카테고리를 추출해주세요. 관련된 정보가 없다면 'null'로 표기해주세요.
        """
        
        # ChatGPT Chat API 호출
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": prompt}],
            max_tokens=300,
            temperature=0.5
        )
        
        output = response['choices'][0]['message']['content'].strip().split("\n")
        
        # 각 필드에 대한 정보를 추출하는 변수 설정
        extracted_data = {
            "expiration_start_date": None,
            "expiration_end_date": None,
            "room_type": None,
            "head_count": None,
            "shipping_fee": 0,  # 기본값은 0
            "transaction_method": None,
            "market_price": None,
            "options": None,
            "parking": None,
            "check_in_time": None,
            "check_out_time": None,
            "sub_category": None
        }

        # 각 항목을 추출해서 딕셔너리에 채우기
        for line in output:
            if ":" in line:
                key_value = line.split(":", 1)
                key = key_value[0].strip()
                value = key_value[1].strip() if len(key_value) > 1 else None
            else:
                key = None
                value = None
            
            if key and value:
                if "시작 날짜" in key:
                    extracted_data["expiration_start_date"] = value
                elif "종료 날짜" in key:
                    extracted_data["expiration_end_date"] = value
                elif "객실 타입" in key:
                    extracted_data["room_type"] = value
                elif "인원 수" in key:
                    extracted_data["head_count"] = int(value) if value.isdigit() else None
                elif "배송비" in key:
                    extracted_data["shipping_fee"] = int(value) if value.isdigit() else 0
                elif "거래 유형" in key:
                    extracted_data["transaction_method"] = value
                elif "시중 가격" in key:
                    price = value.replace("만원", "").strip()
                    extracted_data["market_price"] = int(price) * 10000 if price and price.isdigit() else None
                elif "서비스 옵션" in key:
                    extracted_data["options"] = value
                elif "주차" in key:
                    extracted_data["parking"] = True if "있음" in value else False
                elif "체크인 시간" in key:
                    extracted_data["check_in_time"] = value
                elif "체크아웃 시간" in key:
                    extracted_data["check_out_time"] = value
                elif "세부 카테고리" in key:
                    extracted_data["sub_category"] = value

        # 추출된 데이터를 리스트에 추가
        extracted_data_list.append(extracted_data)

    # 추출된 데이터 리스트 반환
    return extracted_data_list

# 배치 단위로 처리
batch_size = 50
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for i in range(0, len(df), batch_size):
        batch_rows = df.iloc[i:i+batch_size]
        futures.append(executor.submit(extract_hotel_data, batch_rows))
    
    for future in concurrent.futures.as_completed(futures):
        extracted_data_list = future.result()
        for extracted_data in extracted_data_list:
            print(json.dumps(extracted_data, indent=4, ensure_ascii=False))

APIError: The server had an error while processing your request. Sorry about that! {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that!",
    "type": "server_error",
    "param": null,
    "code": null
  }
} 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that!', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Tue, 08 Oct 2024 02:53:18 GMT', 'Content-Type': 'application/json', 'Content-Length': '175', 'Connection': 'keep-alive', 'access-control-expose-headers': 'X-Request-ID', 'openai-organization': 'user-yscrfzcaocjuwuq7rxqol6pi', 'openai-processing-ms': '4924', 'openai-version': '2020-10-01', 'strict-transport-security': 'max-age=31536000; includeSubDomains; preload', 'x-ratelimit-limit-requests': '5000', 'x-ratelimit-limit-tokens': '80000', 'x-ratelimit-remaining-requests': '4998', 'x-ratelimit-remaining-tokens': '77845', 'x-ratelimit-reset-requests': '19ms', 'x-ratelimit

In [ ]:
prompt_df = pd.DataFrame([extracted_data])

In [38]:
import openai
import json
import jsonschema
import concurrent.futures
from jsonschema import validate
import pandas as pd

# JSON 스키마 정의
hotel_schema = {
    "type": "object",
    "properties": {
        "expiration_start_date": {"type": ["string", "null"], "format": "date"},
        "expiration_end_date": {"type": ["string", "null"], "format": "date"},
        "room_type": {"type": ["string", "null"]},
        "head_count": {"type": ["integer", "null"]},
        "shipping_fee": {"type": "integer"},
        "transaction_method": {"type": ["string", "null"]},
        "market_price": {"type": ["integer", "null"]},
        "options": {"type": ["string", "null"]},
        "parking": {"type": ["boolean", "null"]},
        "check_in_time": {"type": ["string", "null"], "format": "time"},
        "check_out_time": {"type": ["string", "null"], "format": "time"},
        "sub_category": {"type": ["string", "null"]}
    },
    "required": ["expiration_start_date", "expiration_end_date", "room_type", "head_count", "shipping_fee", "transaction_method", "market_price", "options", "parking", "check_in_time", "check_out_time", "sub_category"]
}

# ChatGPT API를 통해 정보를 추출하는 함수
def extract_hotel_data(row):
    extracted_data = {
        "expiration_start_date": None,
        "expiration_end_date": None,
        "room_type": None,
        "head_count": None,
        "shipping_fee": 0,  # 기본값은 0
        "transaction_method": None,
        "market_price": None,
        "options": None,
        "parking": None,
        "check_in_time": None,
        "check_out_time": None,
        "sub_category": None
    }
    
    title = row['title']
    description = row['description']
    post_time = row['post_time']
    
    # 채팅 기반 프롬프트 생성
    prompt = f"""
    제목: {title}
    설명: {description}
    업로드 시간: {post_time}

    위 정보를 바탕으로 노쇼 상품에 대한 다음 정보를 추출해주세요:
    
    1. 사용 가능 시작 날짜 (expiration_start_date): 시작 날짜를 추출해주세요. 만약 정확한 날짜가 없이 시간 대명사로 되어 있다면 업로드 시간을 참조하여 계산하여 표기해주세요. 날짜 형식은 '20XX-MM-DD'입니다. 시작 날짜가 없으면 'null'로 표기해주세요.
    2. 사용 가능 마지막 날짜 (expiration_end_date): 종료 날짜를 추출해주세요. 만약 정확한 날짜가 없이 시간 대명사로 되어 있다면 업로드 시간을 참조하여 계산하여 표기해주세요. 날짜 형식은 '20XX-MM-DD'입니다. 마지막 날짜가 없으면 'null'로 표기해주세요.
    3. 객실 타입 (room_type): 호텔 객실 타입을 추출해주세요. 관련된 정보가 없다면 'null'로 표기해주세요.
    4. 사용 인원 수 (head_count): 설명에 언급된 인원 수를 추출해주세요. 인원 수가 없다면 'null'로 표기해주세요.
    5. 배송비 (shipping_fee): 설명에 배송비가 있으면 숫자로 표기해주세요. 없으면 0으로 표기해주세요.
    6. 거래 유형 (transaction_method): 직거래나 택배거래 정보를 추출해주세요. 없으면 'null'로 표기해주세요.
    7. 시중 가격 (market_price): 원래 상품의 가격을 추출해주세요. 가격이 없으면 'null'로 표기해주세요. 숫자 형식으로 표기해주세요. 예: 233000.
    8. 서비스 옵션 (options): 조식, 사우나, 수영장 등의 서비스 옵션이 있는지 확인하고 적어주세요. 없으면 'null'로 표기해주세요.
    9. 주차 여부 (parking): 주차 가능 여부를 확인하고 TRUE 또는 FALSE로 표기해주세요. 없으면 'null'로 표기해주세요.
    10. 체크인 시간 (check_in_time): 체크인 시간을 추출해주세요. 없으면 'null'로 표기해주세요. 시간 형식은 'HH:mm'입니다.
    11. 체크아웃 시간 (check_out_time): 체크아웃 시간을 추출해주세요. 없으면 'null'로 표기해주세요. 시간 형식은 'HH:mm'입니다.
    12. 세부 카테고리 (sub_category): 제목과 설명을 참조하여 호텔,리조트,모텔,펜션,콘도,풀빌라,글램핕,캠핑 등의 카테고리를 추출해주세요. 관련된 정보가 없다면 'null'로 표기해주세요.
    """
    
    # ChatGPT Chat API 호출
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.5
    )
    
    output = response['choices'][0]['message']['content'].strip().split("\n")
    
    # 각 항목을 추출해서 딕셔너리에 채우기
    for line in output:
        if "시작 날짜" in line:
            extracted_data["expiration_start_date"] = line.split(":")[1].strip() if ":" in line else None
        elif "종료 날짜" in line:
            extracted_data["expiration_end_date"] = line.split(":")[1].strip() if ":" in line else None
        elif "객실 타입" in line:
            extracted_data["room_type"] = line.split(":")[1].strip() if ":" in line else None
        elif "인원 수" in line:
            extracted_data["head_count"] = int(line.split(":")[1].strip()) if line.split(":")[1].strip().isdigit() else None
        elif "배송비" in line:
            extracted_data["shipping_fee"] = int(line.split(":")[1].strip()) if line.split(":")[1].strip().isdigit() else 0
        elif "거래 유형" in line:
            extracted_data["transaction_method"] = line.split(":")[1].strip() if ":" in line else None
        elif "시중 가격" in line:
            price = line.split(":")[1].strip() if ":" in line else None
            extracted_data["market_price"] = int(price.replace("만원", "").strip()) * 10000 if price and "만원" in price else None
        elif "서비스 옵션" in line:
            extracted_data["options"] = line.split(":")[1].strip() if ":" in line else None
        elif "주차" in line:
            extracted_data["parking"] = True if "있음" in line else False
        elif "체크인 시간" in line:
            extracted_data["check_in_time"] = line.split(":")[1].strip() if ":" in line else None
        elif "체크아웃 시간" in line:
            extracted_data["check_out_time"] = line.split(":")[1].strip() if ":" in line else None
        elif "세부 카테고리" in line:
            extracted_data["sub_category"] = line.split(":")[1].strip() if ":" in line else None

    return extracted_data

# 멀티쓰레딩을 통한 배치 처리
batch_size = 5
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for i in range(0, len(test_df), batch_size):
        batch_rows = test_df.iloc[i:i+batch_size]
        futures.append(executor.submit(lambda rows: [extract_hotel_data(row) for _, row in rows.iterrows()], batch_rows))
    
    # 결과 출력
    for future in concurrent.futures.as_completed(futures):
        extracted_data_list = future.result()
        for extracted_data in extracted_data_list:
            print(json.dumps(extracted_data, indent=4, ensure_ascii=False))


{
    "expiration_start_date": "null",
    "expiration_end_date": null,
    "room_type": "null",
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": "null",
    "market_price": null,
    "options": "인피니티 수영장 (Infinity pool is mentioned)",
    "parking": false,
    "check_in_time": "null",
    "check_out_time": "null",
    "sub_category": "호텔 (The title mentions 호텔수성, which implies it's a hotel)"
}
{
    "expiration_start_date": "2024-10-02",
    "expiration_end_date": null,
    "room_type": "금연객실",
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": "null",
    "market_price": null,
    "options": "null",
    "parking": false,
    "check_in_time": "20",
    "check_out_time": "12",
    "sub_category": "호텔"
}
{
    "expiration_start_date": "'null'",
    "expiration_end_date": null,
    "room_type": "'null'",
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": "'null'",
    "market_price": null,
    "options": "'null'",
    "p

In [42]:
import openai
import json
import jsonschema
import concurrent.futures
from jsonschema import validate
import pandas as pd

# JSON 스키마 정의
hotel_schema = {
    "type": "object",
    "properties": {
        "expiration_start_date": {"type": ["string", "null"], "format": "date"},
        "expiration_end_date": {"type": ["string", "null"], "format": "date"},
        "room_type": {"type": ["string", "null"]},
        "head_count": {"type": ["integer", "null"]},
        "shipping_fee": {"type": "integer"},
        "transaction_method": {"type": ["string", "null"]},
        "market_price": {"type": ["integer", "null"]},
        "options": {"type": ["string", "null"]},
        "parking": {"type": ["boolean", "null"]},
        "check_in_time": {"type": ["string", "null"], "format": "time"},
        "check_out_time": {"type": ["string", "null"], "format": "time"},
        "sub_category": {"type": ["string", "null"]}
    },
    "required": ["expiration_start_date", "expiration_end_date", "room_type", "head_count", "shipping_fee", "transaction_method", "market_price", "options", "parking", "check_in_time", "check_out_time", "sub_category"]
}

# ChatGPT API를 통해 정보를 추출하는 함수
def extract_hotel_data(row):
    extracted_data = {
        "expiration_start_date": None,
        "expiration_end_date": None,
        "room_type": None,
        "head_count": None,
        "shipping_fee": 0,  # 기본값은 0
        "transaction_method": None,
        "market_price": None,
        "options": None,
        "parking": None,
        "check_in_time": None,
        "check_out_time": None,
        "sub_category": None
    }
    
    title = row['title']
    description = row['description']
    post_time = row['post_time']
    
    # 채팅 기반 프롬프트 생성
    prompt = f"""
    제목: {title}
    설명: {description}
    업로드 시간: {post_time}

    위 정보를 바탕으로 노쇼 상품에 대한 다음 정보를 추출해주세요:
    
    1. 사용 가능 시작 날짜 (expiration_start_date): 시작 날짜를 추출해주세요. 만약 정확한 날짜가 없이 시간 대명사로 되어 있다면 업로드 시간을 참조하여 계산하여 표기해주세요. 날짜 형식은 '20XX-MM-DD'입니다. 시작 날짜가 없으면 'null'로 표기해주세요.
    2. 사용 가능 마지막 날짜 (expiration_end_date): 종료 날짜를 추출해주세요. 만약 정확한 날짜가 없이 시간 대명사로 되어 있다면 업로드 시간을 참조하여 계산하여 표기해주세요. 날짜 형식은 '20XX-MM-DD'입니다. 마지막 날짜가 없으면 'null'로 표기해주세요.
    3. 객실 타입 (room_type): 호텔 객실 타입을 추출해주세요. 관련된 정보가 없다면 'null'로 표기해주세요.
    4. 사용 인원 수 (head_count): 설명에 언급된 인원 수를 추출해주세요. 인원 수가 없다면 'null'로 표기해주세요.
    5. 배송비 (shipping_fee): 설명에 배송비가 있으면 숫자로 표기해주세요. 없으면 0으로 표기해주세요.
    6. 거래 유형 (transaction_method): 직거래나 택배거래 정보를 추출해주세요. 없으면 'null'로 표기해주세요.
    7. 시중 가격 (market_price): 원래 상품의 가격을 추출해주세요. 가격이 없으면 'null'로 표기해주세요. 숫자 형식으로 표기해주세요. 예: 233000.
    8. 서비스 옵션 (options): 조식, 사우나, 수영장 등의 서비스 옵션이 있는지 확인하고 적어주세요. 없으면 'null'로 표기해주세요.
    9. 주차 여부 (parking): 주차 가능 여부를 확인하고 TRUE 또는 FALSE로 표기해주세요. 없으면 'null'로 표기해주세요.
    10. 체크인 시간 (check_in_time): 체크인 시간을 추출해주세요. 없으면 'null'로 표기해주세요. 시간 형식은 'HH:mm'입니다.
    11. 체크아웃 시간 (check_out_time): 체크아웃 시간을 추출해주세요. 없으면 'null'로 표기해주세요. 시간 형식은 'HH:mm'입니다.
    12. 세부 카테고리 (sub_category): 제목과 설명을 참조하여 호텔,리조트,모텔,펜션,콘도,풀빌라,글램핕,캠핑 등의 카테고리를 추출해주세요. 관련된 정보가 없다면 'null'로 표기해주세요.
    """
    
    # ChatGPT Chat API 호출
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.5
    )
    
    output = response['choices'][0]['message']['content'].strip().split("\n")
    
    # 각 항목을 추출해서 딕셔너리에 채우기
    for line in output:
        if "시작 날짜" in line:
            extracted_data["expiration_start_date"] = line.split(":")[1].strip() if ":" in line else None
        elif "종료 날짜" in line:
            extracted_data["expiration_end_date"] = line.split(":")[1].strip() if ":" in line else None
        elif "객실 타입" in line:
            extracted_data["room_type"] = line.split(":")[1].strip() if ":" in line else None
        elif "인원 수" in line:
            extracted_data["head_count"] = int(line.split(":")[1].strip()) if line.split(":")[1].strip().isdigit() else None
        elif "배송비" in line:
            extracted_data["shipping_fee"] = int(line.split(":")[1].strip()) if line.split(":")[1].strip().isdigit() else 0
        elif "거래 유형" in line:
            extracted_data["transaction_method"] = line.split(":")[1].strip() if ":" in line else None
        elif "시중 가격" in line:
            price = line.split(":")[1].strip() if ":" in line else None
            extracted_data["market_price"] = int(price.replace("만원", "").strip()) * 10000 if price and "만원" in price else None
        elif "서비스 옵션" in line:
            extracted_data["options"] = line.split(":")[1].strip() if ":" in line else None
        elif "주차" in line:
            extracted_data["parking"] = True if "있음" in line else False
        elif "체크인 시간" in line:
            extracted_data["check_in_time"] = line.split(":")[1].strip() if ":" in line else None
        elif "체크아웃 시간" in line:
            extracted_data["check_out_time"] = line.split(":")[1].strip() if ":" in line else None
        elif "세부 카테고리" in line:
            extracted_data["sub_category"] = line.split(":")[1].strip() if ":" in line else None

    return extracted_data

# 멀티쓰레딩을 통한 배치 처리
batch_size = 5
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for i in range(0, len(df), batch_size):
        batch_rows = df.iloc[i:i+batch_size]
        futures.append(executor.submit(lambda rows: [extract_hotel_data(row) for _, row in rows.iterrows()], batch_rows))
    
    # 결과 출력
    for future in concurrent.futures.as_completed(futures):
        extracted_data_list = future.result()
        for extracted_data in extracted_data_list:
            print(json.dumps(extracted_data, indent=4, ensure_ascii=False))

IndexError: list index out of range

In [36]:
import openai
import json
import jsonschema
from jsonschema import validate

# JSON 스키마 정의
hotel_schema = {
    "type": "object",
    "properties": {
        "expiration_start_date": {"type": ["string", "null"], "format": "date"},
        "expiration_end_date": {"type": ["string", "null"], "format": "date"},
        "room_type": {"type": ["string", "null"]},
        "head_count": {"type": ["integer", "null"]},
        "shipping_fee": {"type": "integer"},
        "transaction_method": {"type": ["string", "null"]},
        "market_price": {"type": ["integer", "null"]},
        "options": {"type": ["string", "null"]},
        "parking": {"type": ["boolean", "null"]},
        "check_in_time": {"type": ["string", "null"], "format": "time"},
        "check_out_time": {"type": ["string", "null"], "format": "time"},
        "sub_category": {"type": ["string", "null"]}
    },
    "required": ["expiration_start_date", "expiration_end_date", "room_type", "head_count", "shipping_fee", "transaction_method", "market_price", "options", "parking", "check_in_time", "check_out_time", "sub_category"]
}

# ChatGPT API를 통해 정보를 추출하는 함수
def extract_hotel_data(rows):
    extracted_data_list = []
    
    # 각 행에 대해 반복
    for index, row in rows.iterrows():
        title = row['title']
        description = row['description']
        post_time = row['post_time']
        
        # 채팅 기반 프롬프트 생성
        prompt = f"""
        제목: {title}
        설명: {description}
        업로드 시간: {post_time}

        위 정보를 바탕으로 노쇼 상품에 대한 다음 정보를 추출해주세요:
        
        1. 사용 가능 시작 날짜 (expiration_start_date): 시작 날짜를 추출해주세요. 만약 정확한 날짜가 없이 시간 대명사로 되어 있다면 업로드 시간을 참조하여 계산하여 표기해주세요. 날짜 형식은 '20XX-MM-DD'입니다. 시작 날짜가 없으면 'null'로 표기해주세요.
        2. 사용 가능 마지막 날짜 (expiration_end_date): 종료 날짜를 추출해주세요. 만약 정확한 날짜가 없이 시간 대명사로 되어 있다면 업로드 시간을 참조하여 계산하여 표기해주세요. 날짜 형식은 '20XX-MM-DD'입니다. 마지막 날짜가 없으면 'null'로 표기해주세요.
        3. 객실 타입 (room_type): 호텔 객실 타입을 추출해주세요. 관련된 정보가 없다면 'null'로 표기해주세요.
        4. 사용 인원 수 (head_count): 설명에 언급된 인원 수를 추출해주세요. 인원 수가 없다면 'null'로 표기해주세요.
        5. 배송비 (shipping_fee): 설명에 배송비가 있으면 숫자로 표기해주세요. 없으면 0으로 표기해주세요.
        6. 거래 유형 (transaction_method): 직거래나 택배거래 정보를 추출해주세요. 없으면 'null'로 표기해주세요.
        7. 시중 가격 (market_price): 원래 상품의 가격을 추출해주세요. 가격이 없으면 'null'로 표기해주세요. 숫자 형식으로 표기해주세요. 예: 233000.
        8. 서비스 옵션 (options): 조식, 사우나, 수영장 등의 서비스 옵션이 있는지 확인하고 적어주세요. 없으면 'null'로 표기해주세요.
        9. 주차 여부 (parking): 주차 가능 여부를 확인하고 TRUE 또는 FALSE로 표기해주세요. 없으면 'null'로 표기해주세요.
        10. 체크인 시간 (check_in_time): 체크인 시간을 추출해주세요. 없으면 'null'로 표기해주세요. 시간 형식은 'HH:mm'입니다.
        11. 체크아웃 시간 (check_out_time): 체크아웃 시간을 추출해주세요. 없으면 'null'로 표기해주세요. 시간 형식은 'HH:mm'입니다.
        12. 세부 카테고리 (sub_category): 제목과 설명을 참조하여 호텔,리조트,모텔,펜션,콘도,풀빌라,글램핕,캠핑 등의 카테고리를 추출해주세요. 관련된 정보가 없다면 'null'로 표기해주세요.
        """
        
        # ChatGPT Chat API 호출
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300,
            temperature=0.5
        )
        
        output = response['choices'][0]['message']['content'].strip().split("\n")
        
        # 각 필드에 대한 정보를 추출하는 변수 설정
        extracted_data = {
            "expiration_start_date": None,
            "expiration_end_date": None,
            "room_type": None,
            "head_count": None,
            "shipping_fee": 0,  # 기본값은 0
            "transaction_method": None,
            "market_price": None,
            "options": None,
            "parking": None,
            "check_in_time": None,
            "check_out_time": None,
            "sub_category": None
        }

        # 각 항목을 추출해서 딕셔너리에 채우기
        for line in output:
            if "시작 날짜" in line:
                extracted_data["expiration_start_date"] = line.split(":")[1].strip() if ":" in line else None
            elif "종료 날짜" in line:
                extracted_data["expiration_end_date"] = line.split(":")[1].strip() if ":" in line else None
            elif "객실 타입" in line:
                extracted_data["room_type"] = line.split(":")[1].strip() if ":" in line else None
            elif "인원 수" in line:
                extracted_data["head_count"] = int(line.split(":")[1].strip()) if line.split(":")[1].strip().isdigit() else None
            elif "배송비" in line:
                extracted_data["shipping_fee"] = int(line.split(":")[1].strip()) if line.split(":")[1].strip().isdigit() else 0
            elif "거래 유형" in line:
                extracted_data["transaction_method"] = line.split(":")[1].strip() if ":" in line else None
            elif "시중 가격" in line:
                price = line.split(":")[1].strip() if ":" in line else None
                extracted_data["market_price"] = int(price.replace("만원", "").strip()) * 10000 if price and "만원" in price else None
            elif "서비스 옵션" in line:
                extracted_data["options"] = line.split(":")[1].strip() if ":" in line else None
            elif "주차" in line:
                extracted_data["parking"] = True if "있음" in line else False
            elif "체크인 시간" in line:
                extracted_data["check_in_time"] = line.split(":")[1].strip() if ":" in line else None
            elif "체크아웃 시간" in line:
                extracted_data["check_out_time"] = line.split(":")[1].strip() if ":" in line else None
            elif "세부 카테고리" in line:
                extracted_data["sub_category"] = line.split(":")[1].strip() if ":" in line else None

        # 추출된 데이터를 리스트에 추가
        extracted_data_list.append(extracted_data)

    # 추출된 데이터 리스트 반환
    return extracted_data_list

# 배치 단위로 처리
batch_size = 5
for i in range(0, len(test_df), batch_size):
    batch_rows = test_df.iloc[i:i+batch_size]
    extracted_data_list = extract_hotel_data(batch_rows)

    # JSON 형식으로 출력
    for extracted_data in extracted_data_list:
        print(json.dumps(extracted_data, indent=4, ensure_ascii=False))

{
    "expiration_start_date": "'null'",
    "expiration_end_date": null,
    "room_type": "'투룸 슈페리어 더블'",
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": "직거래 가능, 택배거래 가능 (상황에 따라)",
    "market_price": null,
    "options": "사우나, 수영장",
    "parking": false,
    "check_in_time": "'null'",
    "check_out_time": "'null'",
    "sub_category": "호텔"
}
{
    "expiration_start_date": "'null'",
    "expiration_end_date": null,
    "room_type": "'null'",
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": "'비대면가능'",
    "market_price": null,
    "options": "'수영장, 루프탑, 인피니티풀, 온천, 사우나, 이벤트탕'",
    "parking": false,
    "check_in_time": "'null'",
    "check_out_time": "'null'",
    "sub_category": "'호텔'"
}
{
    "expiration_start_date": "2024-10-02",
    "expiration_end_date": null,
    "room_type": "테라스, 무라타, 풀하우스, 펜트하우스A, 클리퍼듀플렉스",
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": "null",
    "market_price": null,
    "options":

In [37]:
prompt_df=pd.DataFrame([extracted_data])
prompt_df

,expiration_start_date,expiration_end_date,room_type,head_count,shipping_fee,transaction_method,market_price,options,parking,check_in_time,check_out_time,sub_category
0,2024-10-02,None,비지니스룸,None,0,null,None,null,False,15,21,호텔


In [33]:
import openai
import json
import jsonschema
from jsonschema import validate

# JSON 스키마 정의
hotel_schema = {
    "type": "object",
    "properties": {
        "expiration_start_date": { "type": ["string", "null"], "format": "date" },
        "expiration_end_date": { "type": ["string", "null"], "format": "date" },
        "room_type": { "type": ["string", "null"] },
        "head_count": { "type": ["integer", "null"] },
        "shipping_fee": { "type": "integer" },
        "transaction_method": { "type": ["string", "null"] },
        "market_price": { "type": ["integer", "null"] },
        "options": { "type": ["string", "null"] },
        "parking": { "type": ["boolean", "null"] },
        "check_in_time": { "type": ["string", "null"], "format": "time" },
        "check_out_time": { "type": ["string", "null"], "format": "time" },
        "sub_category": { "type": ["string", "null"] }
    },
    "required": ["expiration_start_date", "expiration_end_date", "room_type", "head_count", "shipping_fee", "transaction_method", "market_price", "options", "parking", "check_in_time", "check_out_time", "sub_category"]
}

# ChatGPT API를 통해 정보를 추출하는 함수
def extract_hotel_data(row):
    title = row['title']
    description = row['description']
    post_time = row['post_time']
    
    # 채팅 기반 프롬프트 생성
    prompt = f"""
    제목: {title}
    설명: {description}
    업로드 시간: {post_time}

    위 정보를 바탕으로 노쇼 상품에 대한 다음 정보를 추출해주세요:
    
    1. 사용 가능 시작 날짜 (expiration_start_date): 시작 날짜를 추출해주세요. 만약 정확한 날짜가 없이 시간 대명사로 되어 있다면 업로드 시간을 참조하여 계산하여 표기해주세요. 날짜 형식은 '20XX-MM-DD'입니다. 시작 날짜가 없으면 'null'로 표기해주세요.
    2. 사용 가능 마지막 날짜 (expiration_end_date): 종료 날짜를 추출해주세요. 만약 정확한 날짜가 없이 시간 대명사로 되어 있다면 업로드 시간을 참조하여 계산하여 표기해주세요. 날짜 형식은 '20XX-MM-DD'입니다. 마지막 날짜가 없으면 'null'로 표기해주세요.
    3. 객실 타입 (room_type): 호텔 객실 타입을 추출해주세요. 관련된 정보가 없다면 'null'로 표기해주세요.
    4. 사용 인원 수 (head_count): 설명에 언급된 인원 수를 추출해주세요. 인원 수가 없다면 'null'로 표기해주세요.
    5. 배송비 (shipping_fee): 설명에 배송비가 있으면 숫자로 표기해주세요. 없으면 0으로 표기해주세요.
    6. 거래 유형 (transaction_method): 직거래나 택배거래 정보를 추출해주세요. 없으면 'null'로 표기해주세요.
    7. 시중 가격 (market_price): 원래 상품의 가격을 추출해주세요. 가격이 없으면 'null'로 표기해주세요. 숫자 형식으로 표기해주세요. 예: 233000.
    8. 서비스 옵션 (options): 조식, 사우나, 수영장 등의 서비스 옵션이 있는지 확인하고 적어주세요. 없으면 'null'로 표기해주세요.
    9. 주차 여부 (parking): 주차 가능 여부를 확인하고 TRUE 또는 FALSE로 표기해주세요. 없으면 'null'로 표기해주세요.
    10. 체크인 시간 (check_in_time): 체크인 시간을 추출해주세요. 없으면 'null'로 표기해주세요. 시간 형식은 'HH:mm'입니다.
    11. 체크아웃 시간 (check_out_time): 체크아웃 시간을 추출해주세요. 없으면 'null'로 표기해주세요. 시간 형식은 'HH:mm'입니다.
    12. 세부 카테고리 (sub_category): 제목과 설명을 참조하여 호텔,리조트,모텔,펜션,콘도,풀빌라,글램핕,캠핑 등의 카테고리를 추출해주세요. 관련된 정보가 없다면 'null'로 표기해주세요.
    """
    
    # ChatGPT Chat API 호출
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.5
    )
    
    output = response['choices'][0]['message']['content'].strip().split("\n")
    
    # 각 필드에 대한 정보를 추출하는 변수 설정
    extracted_data = {
        "expiration_start_date": None,
        "expiration_end_date": None,
        "room_type": None,
        "head_count": None,
        "shipping_fee": 0,  # 기본값은 0
        "transaction_method": None,
        "market_price": None,
        "options": None,
        "parking": None,
        "check_in_time": None,
        "check_out_time": None,
        "sub_category": None
    }

    # 각 항목을 추출해서 딕셔너리에 채우기
    # 각 항목을 추출해서 딕셔너리에 채우기
    # 각 항목을 추출해서 딕셔너리에 채우기
    for line in output:
        if "시작 날짜" in line:
            extracted_data["expiration_start_date"] = line.split(":")[1].strip() if ":" in line else None
        elif "종료 날짜" in line:
            extracted_data["expiration_end_date"] = line.split(":")[1].strip() if ":" in line else None
        elif "객실 타입" in line:
            extracted_data["room_type"] = line.split(":")[1].strip() if ":" in line else None
        elif "인원 수" in line:
            extracted_data["head_count"] = int(line.split(":")[1].strip()) if line.split(":")[1].strip().isdigit() else None
        elif "배송비" in line:
            extracted_data["shipping_fee"] = int(line.split(":")[1].strip()) if line.split(":")[1].strip().isdigit() else 0
        elif "거래 유형" in line:
            extracted_data["transaction_method"] = line.split(":")[1].strip() if ":" in line else None
        elif "시중 가격" in line:
            price = line.split(":")[1].strip() if ":" in line else None
            extracted_data["market_price"] = int(price.replace("만원", "").strip()) * 10000 if price and "만원" in price else None
        elif "서비스 옵션" in line:
            extracted_data["options"] = line.split(":")[1].strip() if ":" in line else None
        elif "주차" in line:
            extracted_data["parking"] = True if "있음" in line else False
        elif "체크인 시간" in line:
            extracted_data["check_in_time"] = line.split(":")[1].strip() if ":" in line else None
        elif "체크아웃 시간" in line:
            extracted_data["check_out_time"] = line.split(":")[1].strip() if ":" in line else None
        elif "세부 카테고리" in line:
            extracted_data["sub_category"] = line.split(":")[1].strip() if ":" in line else None


    # JSON 스키마 검증
    try:
        validate(instance=extracted_data, schema=hotel_schema)
        print("JSON 데이터가 스키마를 통과했습니다.")
    except jsonschema.exceptions.ValidationError as ve:
        print("JSON 데이터 검증 실패:", ve)

    # 반환된 데이터 확인
    return extracted_data

# 데이터프레임의 각 행에 대해 데이터 추출 및 검증
for index, row in test_df.iterrows():
    extracted_data = extract_hotel_data(row)
    
    # JSON 형식으로 출력
    print(json.dumps(extracted_data, indent=4, ensure_ascii=False))
    print(extracted_data)

JSON 데이터가 스키마를 통과했습니다.
{
    "expiration_start_date": "'null'",
    "expiration_end_date": null,
    "room_type": "'투룸 슈페리어 더블'",
    "head_count": 6,
    "shipping_fee": 0,
    "transaction_method": "'직거래'",
    "market_price": null,
    "options": "'사우나, 수영장'",
    "parking": false,
    "check_in_time": "'null'",
    "check_out_time": "'null'",
    "sub_category": "'호텔'"
}
{'expiration_start_date': "'null'", 'expiration_end_date': None, 'room_type': "'투룸 슈페리어 더블'", 'head_count': 6, 'shipping_fee': 0, 'transaction_method': "'직거래'", 'market_price': None, 'options': "'사우나, 수영장'", 'parking': False, 'check_in_time': "'null'", 'check_out_time': "'null'", 'sub_category': "'호텔'"}
JSON 데이터가 스키마를 통과했습니다.
{
    "expiration_start_date": "'null'",
    "expiration_end_date": null,
    "room_type": "'null'",
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": "'비대면가능'",
    "market_price": null,
    "options": "'수영장, 루프탑, 인피니티풀, 온천, 사우나, 이벤트탕'",
    "parking": false,
    "check

In [26]:
# JSON 스키마 정의
hotel_schema = {
    "type": "object",
    "properties": {
        "expiration_start_date": { "type": ["string", "null"], "format": "date" },
        "expiration_end_date": { "type": ["string", "null"], "format": "date" },
        "room_type": { "type": ["string", "null"] },
        "head_count": { "type": ["integer", "null"] },
        "shipping_fee": { "type": "integer" },
        "transaction_method": { "type": ["string", "null"] },
        "market_price": { "type": ["integer", "null"] },
        "options": { "type": ["string", "null"] },
        "parking": { "type": ["boolean", "null"] },
        "check_in_time": { "type": ["string", "null"], "format": "time" },
        "check_out_time": { "type": ["string", "null"], "format": "time" },
        "sub_category": { "type": ["string", "null"] }
    },
    "required": ["expiration_start_date", "expiration_end_date", "room_type", "head_count", "shipping_fee", "transaction_method", "market_price", "options", "parking", "check_in_time", "check_out_time", "sub_category"]
}

# ChatGPT API를 통해 정보를 추출하는 함수
def extract_hotel_data(row):
    title = row['title']
    description = row['description']
    post_time = row['post_time']
    
    prompt = f"""
    제목: {title}
    설명: {description}
    업로드 시간: {post_time}

    위 정보를 바탕으로 노쇼 상품에 대한 다음 정보를 추출해주세요:
    
    1. 사용 가능 시작 날짜 (expiration_start_date): 시작 날짜를 추출해주세요. 만약 정확한 날짜가 없이 시간 대명사로 되어 있다면 업로드 시간을 참조하여 계산하여 표기해주세요. 날짜 형식은 '20XX-MM-DD'입니다. 시작 날짜가 없으면 'null'로 표기해주세요.
    2. 사용 가능 마지막 날짜 (expiration_end_date): 종료 날짜를 추출해주세요. 만약 정확한 날짜가 없이 시간 대명사로 되어 있다면 업로드 시간을 참조하여 계산하여 표기해주세요. 날짜 형식은 '20XX-MM-DD'입니다. 마지막 날짜가 없으면 'null'로 표기해주세요.
    3. 객실 타입 (room_type): 호텔 객실 타입을 추출해주세요. 관련된 정보가 없다면 'null'로 표기해주세요.
    4. 사용 인원 수 (head_count): 설명에 언급된 인원 수를 추출해주세요. 인원 수가 없다면 'null'로 표기해주세요.
    5. 배송비 (shipping_fee): 설명에 배송비가 있으면 숫자로 표기해주세요. 없으면 0으로 표기해주세요.
    6. 거래 유형 (transaction_method): 직거래나 택배거래 정보를 추출해주세요. 없으면 'null'로 표기해주세요.
    7. 시중 가격 (market_price): 원래 상품의 가격을 추출해주세요. 가격이 없으면 'null'로 표기해주세요. 숫자 형식으로 표기해주세요. 예: 233000.
    8. 서비스 옵션 (options): 조식, 사우나, 수영장 등의 서비스 옵션이 있는지 확인하고 적어주세요. 없으면 'null'로 표기해주세요.
    9. 주차 여부 (parking): 주차 가능 여부를 확인하고 TRUE 또는 FALSE로 표기해주세요. 없으면 'null'로 표기해주세요.
    10. 체크인 시간 (check_in_time): 체크인 시간을 추출해주세요. 없으면 'null'로 표기해주세요.
    11. 체크아웃 시간 (check_out_time): 체크아웃 시간을 추출해주세요. 없으면 'null'로 표기해주세요.
    12. 세부 카테고리 (sub_category): 제목과 설명을 참조하여 호텔,리조트,모텔,펜션,콘도,풀빌라,글램핕,캠핑 등의 카테고리를 추출해주세요. 관련된 정보가 없다면 'null'로 표기해주세요.
    """
    
    response = openai.Completion.create(
        engine="gpt-4",
        prompt=prompt,
        max_tokens=300,
        temperature=0.5
    )
    
    output = response.choices[0].text.strip().split("\n")
    
    # 각 필드에 대한 정보를 추출하는 변수 설정
    extracted_data = {
        "expiration_start_date": None,
        "expiration_end_date": None,
        "room_type": None,
        "head_count": None,
        "shipping_fee": 0,  # 기본값은 0
        "transaction_method": None,
        "market_price": None,
        "options": None,
        "parking": None,
        "check_in_time": None,
        "check_out_time": None,
        "sub_category": None
    }

    # 각 항목을 추출해서 딕셔너리에 채우기
    for line in output:
        if "시작 날짜" in line:
            extracted_data["expiration_start_date"] = line.split(":")[1].strip()
        elif "종료 날짜" in line:
            extracted_data["expiration_end_date"] = line.split(":")[1].strip()
        elif "객실 타입" in line:
            extracted_data["room_type"] = line.split(":")[1].strip()
        elif "인원 수" in line:
            extracted_data["head_count"] = int(line.split(":")[1].strip()) if line.split(":")[1].strip().isdigit() else None
        elif "배송비" in line:
            extracted_data["shipping_fee"] = int(line.split(":")[1].strip()) if line.split(":")[1].strip().isdigit() else 0
        elif "거래 유형" in line:
            extracted_data["transaction_method"] = line.split(":")[1].strip()
        elif "시중 가격" in line:
            price = line.split(":")[1].strip()
            extracted_data["market_price"] = int(price.replace("만원", "").strip()) * 10000 if "만원" in price else None
        elif "서비스 옵션" in line:
            extracted_data["options"] = line.split(":")[1].strip()
        elif "주차" in line:
            extracted_data["parking"] = True if "있음" in line else False
        elif "체크인 시간" in line:
            extracted_data["check_in_time"] = line.split(":")[1].strip()
        elif "체크아웃 시간" in line:
            extracted_data["check_out_time"] = line.split(":")[1].strip()
        elif "세부 카테고리" in line:
            extracted_data["sub_category"] = line.split(":")[1].strip()
    
    # JSON 스키마 검증
    try:
        validate(instance=extracted_data, schema=hotel_schema)
        print("JSON 데이터가 스키마를 통과했습니다.")
    except jsonschema.exceptions.ValidationError as ve:
        print("JSON 데이터 검증 실패:", ve)

    # 반환된 데이터 확인
    return extracted_data

# 데이터프레임의 각 행에 대해 데이터 추출 및 검증
for index, row in df.iterrows():
    extracted_data = extract_hotel_data(row)
    
    # JSON 형식으로 출력
    print(json.dumps(extracted_data, indent=4, ensure_ascii=False))

InvalidRequestError: This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?

In [ ]:
import pandas as pd
import jsonschema
from jsonschema import validate

# JSON Schema 정의 (title, post_time, description을 참조)
schema = {
    "type": "object",
    "properties": {
        "expiration_start_date": {"type": ["string", "null"], "description": "시작일"},
        "expiration_end_date": {"type": ["string", "null"], "description": "종료일"},
        "room_type": {"type": ["string", "null"], "description": "방 종류"},
        "head_count": {"type": ["integer", "null"], "description": "인원수"},
        "shipping_fee": {"type": ["integer", "null"], "description": "배송비"},
        "transaction_method": {"type": ["string", "null"], "description": "거래 방법"},
        "market_price": {"type": ["integer", "null"], "description": "시장 가격"},
        "options": {"type": ["string", "null"], "description": "옵션 정보"},
        "parking": {"type": ["string", "null"], "description": "주차 여부"},
        "check_in_time": {"type": ["string", "null"], "description": "체크인 시간"},
        "check_out_time": {"type": ["string", "null"], "description": "체크아웃 시간"},
        "sub_category": {"type": ["string", "null"], "description": "하위 카테고리"}
    },
    "required": ["expiration_start_date", "expiration_end_date"]
}

# ChatGPT API를 통해 날짜 정보를 추출하는 함수
def get_expiration_dates(title, description, post_time):
    prompt = f"""
    제목: {title}
    설명: {description}
    업로드 시간: {post_time}
    
    위 정보를 바탕으로 노쇼 상품의 사용 가능 시작 날짜와 종료 날짜를 추출해주세요.
    (시작일과 종료일을 '20XX-MM-DD' 형식으로 출력하며, '오늘'과 '내일'은 현재 날짜와 내일로 변환하세요. 명확한 날짜가 없다면 'null'로 표기해주세요.)
    """
    
    response = openai.Completion.create(
        engine="gpt-4",
        prompt=prompt,
        max_tokens=150,
        temperature=0.5
    )
    
    output = response.choices[0].text.strip().split("\n")
    
    # 날짜를 추출해서 반환 (예: "2024-10-02", "2024-10-03" 등)
    expiration_start_date = None
    expiration_end_date = None
    
    for line in output:
        if "시작일" in line:
            expiration_start_date = line.split(":")[1].strip()
        if "종료일" in line:
            expiration_end_date = line.split(":")[1].strip()
    
    return expiration_start_date, expiration_end_date

# 예시 데이터
title = "호텔 숙박권 판매"
description = "이 숙박권은 10월 2일부터 10월 3일까지 유효합니다. 오늘이 마지막 날입니다."
post_time = "2024-10-01"

# 날짜 정보 추출
expiration_start_date, expiration_end_date = get_expiration_dates(title, description, post_time)

# JSON 스키마에 맞는 데이터 구조화
row_data = {
    "expiration_start_date": expiration_start_date,
    "expiration_end_date": expiration_end_date,
    "room_type": "호텔",
    "head_count": None,
    "shipping_fee": None,
    "transaction_method": "직거래",
    "market_price": None,
    "options": None,
    "parking": None,
    "check_in_time": None,
    "check_out_time": None,
    "sub_category": "호텔"
}

# JSON 스키마로 데이터 검증
try:
    validate(instance=row_data, schema=schema)
    print(json.dumps(row_data, indent=4, ensure_ascii=False))
except jsonschema.exceptions.ValidationError as e:
    print(f"Validation error: {e}")

In [25]:
test_df

,platform,original_link,post_time,title,view_count,like_count,price,images,description,category,status,seller_location,title_nouns,seller_location_1
0,당근마켓,https://www.daangn.com/articles/832373103,2024-10-02 12:53:31.443,속초 체스터톤스 호텔,310,7,170000,https://img.kr.gcp-karroter.net/origin/article...,속초 체스터톤스 숙박권입니다.\n3매구요~\n원하는 매수 만큼 가능하세요.\n투룸 ...,티켓/교환권,판매중,부천시 여월동,"[속초, 체스터, 톤스, 호텔]",부천시
1,당근마켓,https://www.daangn.com/articles/840996796,2024-10-02 12:52:31.443,수성호텔 vip 티켓 1장,4,0,34000,https://img.kr.gcp-karroter.net/origin/article...,"수성호텔 호텔수성 vip 티켓 (수영장,루프탑,인피니티풀, 온천, 사우나, 이벤트...",티켓/교환권,판매중,북구 매천동,"[수성, 호텔, 티켓, 장]",북구
2,당근마켓,https://www.daangn.com/articles/840996562,2024-10-02 12:52:31.443,"아난티 리조트호텔 가평, 코브, 빌라쥬드, 남해, 제주 숙박권 양도",12,0,540000,https://img.kr.gcp-karroter.net/origin/article...,"남해 빼고 (평일)일-목 체크인 가능, 금토 불가.\n남해 주말권만 금,토 체크인 ...",티켓/교환권,판매중,남구 사직동,"[아난, 티, 리조트, 호텔, 가평, 코브, 빌라, 쥬드, 남해, 제주, 숙박, 양도]",남구
3,당근마켓,https://www.daangn.com/articles/840986215,2024-10-02 12:35:31.443,파타야 힐튼호텔 디럭스룸 10/12~13,21,0,400000,https://img.kr.gcp-karroter.net/origin/article...,연락주세요,티켓/교환권,판매중,수원시 영통구 영통동,"[파타야, 힐튼, 호텔, 디럭스, 룸]",수원시
4,당근마켓,https://www.daangn.com/articles/840984922,2024-10-02 12:33:31.443,글램 호텔 숙박,57,0,60000,https://img.kr.gcp-karroter.net/origin/article...,경희대 글램 호텔 숙박권 판매합니다.\n연락주세요:),티켓/교환권,판매중,종로구 창신제2동,"[글램, 호텔, 숙박]",종로구
5,당근마켓,https://www.daangn.com/articles/840984846,2024-10-02 12:33:31.443,호텔수성 인피니티 수영장 티켓 3장 사봐요!,29,2,100000,https://img.kr.gcp-karroter.net/origin/article...,연락부탁드려용\n답늦을수도있어요!,티켓/교환권,판매중,북구 산격동,"[호텔, 수성, 인피니티, 수영장, 티켓, 장]",북구
6,당근마켓,https://www.daangn.com/articles/840983318,2024-10-02 12:31:31.443,글램 호텔,42,1,60000,https://img.kr.gcp-karroter.net/origin/article...,10월 2일~3일\n오늘 숙박입니다.\n2일 20시 입실\n3일 12시 퇴실\n경희...,티켓/교환권,판매중,광진구 군자동,"[글램, 호텔]",광진구
7,당근마켓,https://www.daangn.com/articles/840983146,2024-10-02 12:30:31.443,주문진 호텔 양도합니다,188,5,80000,https://img.kr.gcp-karroter.net/origin/article...,사정이 생겨서 못가게 되서 양도합니다.,티켓/교환권,판매중,원주시 단구동,"[주문진, 호텔, 니]",원주시
8,당근마켓,https://www.daangn.com/articles/840982152,2024-10-02 12:29:31.443,속초 체스터톤스호텔 금일숙박팔아요,90,8,40000,https://img.kr.gcp-karroter.net/origin/article...,2024년10월2일 금일.숙박권 팝니다\n원룸스탠다드더블룸,티켓/교환권,판매중,동대문구 장안동,"[속초, 체스터, 톤스, 호텔, 금일, 숙박]",동대문구
9,당근마켓,https://www.daangn.com/articles/840979733,2024-10-02 12:25:31.443,하남 씨엘 호텔 대실,51,0,30000,https://img.kr.gcp-karroter.net/origin/article...,하남 씨엘 CL 호텔 오늘 10월 2일 수요일 6시간 대실 (오 후 3시부터 9시까...,티켓/교환권,판매중,강동구 강일동,"[하남, 씨엘, 호텔]",강동구


In [24]:
pd.DataFrame(extracted_fields)

,expiration_start_date,expiration_end_date,room_type,head_count,shipping_fee,transaction_method,market_price,options,parking,check_in_time,check_out_time,sub_category
0,2024-10-02,2024-10-03,호텔,None,None,None,None,None,None,20:00,12:00,None
1,None,None,호텔,None,None,None,None,None,None,20:00,12:00,None
2,2024-10-02,2024-10-03,호텔,None,None,None,None,None,None,20:00,12:00,None
3,None,None,호텔,None,None,None,None,None,None,20:00,12:00,None
4,2024-10-02,2024-10-03,호텔,None,None,None,None,None,None,20:00,12:00,None
5,None,None,호텔,None,None,None,None,None,None,20:00,12:00,None
6,None,None,호텔,None,None,None,None,None,None,20:00,12:00,None
7,None,None,호텔,None,None,None,None,None,None,20:00,12:00,None
8,2024-10-02,2024-10-03,호텔,None,None,None,None,None,None,20:00,12:00,None
9,None,None,호텔,None,None,None,None,None,None,20:00,12:00,None


In [22]:
schema = {
    "type": "object",
    "properties": {
        "expiration_start_date": {"type": ["string", "null"], "description": "시작일"},
        "expiration_end_date": {"type": ["string", "null"], "description": "종료일"},
        "room_type": {"type": ["string", "null"], "description": "방 종류"},
        "head_count": {"type": ["integer", "null"], "description": "인원수"},
        "shipping_fee": {"type": ["integer", "null"], "description": "배송비"},
        "transaction_method": {"type": ["string", "null"], "description": "거래 방법"},
        "market_price": {"type": ["integer", "null"], "description": "시장 가격"},
        "options": {"type": ["string", "null"], "description": "옵션 정보"},
        "parking": {"type": ["string", "null"], "description": "주차 여부"},
        "check_in_time": {"type": ["string", "null"], "description": "체크인 시간"},
        "check_out_time": {"type": ["string", "null"], "description": "체크아웃 시간"},
        "sub_category": {"type": ["string", "null"], "description": "하위 카테고리"}
    },
    "required": ["expiration_start_date", "expiration_end_date"]
}

# 데이터 필드를 추출하는 함수
def extract_fields(df):
    extracted_data = []
    
    for index, row in df.iterrows():
        row_data = {
            "expiration_start_date": None,  # title이나 description을 바탕으로 값 추출
            "expiration_end_date": None,    # title이나 description을 바탕으로 값 추출
            "room_type": None,
            "head_count": None,
            "shipping_fee": None,
            "transaction_method": None,
            "market_price": None,
            "options": None,
            "parking": None,
            "check_in_time": None,
            "check_out_time": None,
            "sub_category": None
        }
        
        # title을 참조해서 데이터 추출 예시
        if "호텔" in row['title']:
            row_data['room_type'] = "호텔"
        
        # description을 참조해서 정보 추출
        if "숙박권" in row['description']:
            row_data['expiration_start_date'] = "2024-10-02"
            row_data['expiration_end_date'] = "2024-10-03"
        
        # post_time을 참조해서 시간 처리 예시
        row_data['check_in_time'] = "20:00"
        row_data['check_out_time'] = "12:00"

        # schema에 맞는 유효성 검사
        try:
            validate(instance=row_data, schema=schema)
            extracted_data.append(row_data)
        except jsonschema.exceptions.ValidationError as e:
            print(f"Validation error for row {index}: {e}")
    
    return extracted_data

# 결과 출력
extracted_fields = extract_fields(test_df)
print(json.dumps(extracted_fields, indent=4, ensure_ascii=False))

[
    {
        "expiration_start_date": "2024-10-02",
        "expiration_end_date": "2024-10-03",
        "room_type": "호텔",
        "head_count": null,
        "shipping_fee": null,
        "transaction_method": null,
        "market_price": null,
        "options": null,
        "parking": null,
        "check_in_time": "20:00",
        "check_out_time": "12:00",
        "sub_category": null
    },
    {
        "expiration_start_date": null,
        "expiration_end_date": null,
        "room_type": "호텔",
        "head_count": null,
        "shipping_fee": null,
        "transaction_method": null,
        "market_price": null,
        "options": null,
        "parking": null,
        "check_in_time": "20:00",
        "check_out_time": "12:00",
        "sub_category": null
    },
    {
        "expiration_start_date": "2024-10-02",
        "expiration_end_date": "2024-10-03",
        "room_type": "호텔",
        "head_count": null,
        "shipping_fee": null,
        "transaction_me

In [ ]:
import requests
import pandas as pd

# 여기에 GPT-4 API 키 입력
API_KEY = "your_gpt_4_api_key"

def extract_fields_from_row(row):
    # API 요청을 위한 데이터 준비
    prompt = f"""
    데이터: {row['description']}
    다음 필드들을 추출해 주세요:
    - 숙박 시작일 (expiration_start_date)
    - 숙박 종료일 (expiration_end_date)
    - 방 종류 (room_type)
    - 인원수 (head_count)
    - 배송비 (shipping_fee)
    - 거래 방법 (transaction_method)
    - 가격 (market_price)
    - 옵션 (options)
    - 주차 여부 (parking)
    - 체크인 시간 (check_in_time)
    - 체크아웃 시간 (check_out_time)
    - 하위 카테고리 (sub_category)
    """

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "gpt-4",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 500
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=data)
    
    if response.status_code == 200:
        response_data = response.json()
        return response_data['choices'][0]['message']['content']
    else:
        return None

def process_new_data(df):
    extracted_data = []
    for _, row in df.iterrows():
        extracted_fields = extract_fields_from_row(row)
        if extracted_fields:
            extracted_data.append(extracted_fields)
        else:
            extracted_data.append({field: None for field in df.columns})
    
    # 새로 추출된 데이터로 DataFrame 업데이트
    extracted_df = pd.DataFrame(extracted_data, columns=df.columns)
    return extracted_df

# 사용 예시 (새로운 데이터가 크롤링되어 `new_df`에 추가됨)
new_df = pd.DataFrame({
    'description': [
        '속초 체스터톤스 호텔 숙박권 3매, 원하는 매수 가능',
        '수성호텔 vip 티켓 1장, 수영장, 루프탑 포함',
        '아난티 리조트호텔 숙박권 양도, 평일 체크인 가능'
    ]
})

# 새로 추가된 데이터 처리 및 필드 추출
processed_data = process_new_data(new_df)

# 처리된 결과 출력
print(processed_data)


In [20]:
# JSON 스키마 정의
schema = {
    "type": "object",
    "properties": {
        "expiration_start_date": {
            "anyOf": [
                {"type": "string", "format": "date"},
                {"type": "null"}
            ],
            "description": "노쇼 상품의 사용 가능 시작 날짜"
        },
        "expiration_end_date": {
            "anyOf": [
                {"type": "string", "format": "date"},
                {"type": "null"}
            ],
            "description": "노쇼 상품의 사용 가능 마지막 날짜"
        },
        "room_type": {
            "anyOf": [
                {"type": "string"},
                {"type": "null"}
            ],
            "description": "객실 타입"
        },
        "head_count": {
            "anyOf": [
                {"type": "integer"},
                {"type": "null"}
            ],
            "description": "사용 인원 수"
        },
        "shipping_fee": {
            "anyOf": [
                {"type": "integer"},
                {"type": "null"}
            ],
            "description": "배송비"
        },
        "transaction_method": {
            "anyOf": [
                {"type": "string"},
                {"type": "null"}
            ],
            "description": "거래 유형 (직거래, 택배거래)"
        },
        "market_price": {
            "anyOf": [
                {"type": "integer"},
                {"type": "null"}
            ],
            "description": "시중 가격"
        },
        "options": {
            "anyOf": [
                {"type": "string"},
                {"type": "null"}
            ],
            "description": "호텔 서비스 옵션 (조식, 수영장 등)"
        },
        "parking": {
            "anyOf": [
                {"type": "boolean"},
                {"type": "null"}
            ],
            "description": "주차 가능 여부"
        },
        "check_in_time": {
            "anyOf": [
                {"type": "string", "pattern": "^([0-1]?[0-9]|2[0-3]):[0-5][0-9]$"},
                {"type": "null"}
            ],
            "description": "체크인 시간 (HH:mm 형식)"
        },
        "check_out_time": {
            "anyOf": [
                {"type": "string", "pattern": "^([0-1]?[0-9]|2[0-3]):[0-5][0-9]$"},
                {"type": "null"}
            ],
            "description": "체크아웃 시간 (HH:mm 형식)"
        },
        "sub_category": {
            "anyOf": [
                {"type": "string"},
                {"type": "null"}
            ],
            "description": "세부 카테고리 (호텔, 리조트, 펜션/콘도 등)"
        }
    },
    "required": [
        "expiration_start_date",
        "expiration_end_date",
        "room_type",
        "head_count",
        "shipping_fee",
        "transaction_method",
        "market_price",
        "options",
        "parking",
        "check_in_time",
        "check_out_time"
        "sub_category"
    ]
}

# 필드 추출 함수
def extract_fields(df):
    extracted_data_list = []

    # DataFrame의 각 행을 처리
    for _, row in df.iterrows():
        row_data = row.to_dict()

        # 각 필드가 None인 경우 기본 값을 설정 (선택사항)
        row_data["expiration_start_date"] = row_data.get("expiration_start_date", None)
        row_data["expiration_end_date"] = row_data.get("expiration_end_date", None)
        row_data["room_type"] = row_data.get("room_type", None)
        row_data["head_count"] = row_data.get("head_count", None)
        row_data["shipping_fee"] = row_data.get("shipping_fee", None)
        row_data["transaction_method"] = row_data.get("transaction_method", None)
        row_data["market_price"] = row_data.get("market_price", None)
        row_data["options"] = row_data.get("options", None)
        row_data["parking"] = row_data.get("parking", None)
        row_data["check_in_time"] = row_data.get("check_in_time", None)
        row_data["check_out_time"] = row_data.get("check_out_time", None)
        row_data["sub_category"] = row_data.get("sub_category", None)

        # 스키마 검증
        try:
            validate(instance=row_data, schema=schema)
        except ValidationError as e:
            raise ValueError(f"Invalid data: {e.message}")

        # 필드 추출
        extracted_data = {
            "expiration_start_date": row_data.get("expiration_start_date"),
            "expiration_end_date": row_data.get("expiration_end_date"),
            "room_type": row_data.get("room_type"),
            "head_count": row_data.get("head_count"),
            "shipping_fee": row_data.get("shipping_fee"),
            "transaction_method": row_data.get("transaction_method"),
            "market_price": row_data.get("market_price"),
            "options": row_data.get("options"),
            "parking": row_data.get("parking"),
            "check_in_time": row_data.get("check_in_time"),
            "check_out_time": row_data.get("check_out_time"),
            "sub_category": row_data.get("sub_category")
        }

        extracted_data_list.append(extracted_data)

    return extracted_data_list

# GPT-4 호출 함수
def call_gpt4(data):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Please process this data: {data}"}
            ]
        )
        return response.choices[0].message['content']
    except Exception as e:
        raise ValueError(f"Error calling GPT-4: {str(e)}")

# 예시로 GPT-4 호출 및 데이터 추출 실행하기
if __name__ == '__main__':
    extracted_data = extract_fields(test_df)
    gpt_response = call_gpt4(extracted_data)
    print("Extracted Data:", extracted_data)
    print("GPT-4 Response:", gpt_response)


Extracted Data: [{'expiration_start_date': None, 'expiration_end_date': None, 'room_type': None, 'head_count': None, 'shipping_fee': None, 'transaction_method': None, 'market_price': None, 'options': None, 'parking': None, 'check_in_time': None, 'check_out_time': None, 'sub_category': None}, {'expiration_start_date': None, 'expiration_end_date': None, 'room_type': None, 'head_count': None, 'shipping_fee': None, 'transaction_method': None, 'market_price': None, 'options': None, 'parking': None, 'check_in_time': None, 'check_out_time': None, 'sub_category': None}, {'expiration_start_date': None, 'expiration_end_date': None, 'room_type': None, 'head_count': None, 'shipping_fee': None, 'transaction_method': None, 'market_price': None, 'options': None, 'parking': None, 'check_in_time': None, 'check_out_time': None, 'sub_category': None}, {'expiration_start_date': None, 'expiration_end_date': None, 'room_type': None, 'head_count': None, 'shipping_fee': None, 'transaction_method': None, 'mark

In [21]:
test_df

,platform,original_link,post_time,title,view_count,like_count,price,images,description,category,status,seller_location,title_nouns,seller_location_1
0,당근마켓,https://www.daangn.com/articles/832373103,2024-10-02 12:53:31.443,속초 체스터톤스 호텔,310,7,170000,https://img.kr.gcp-karroter.net/origin/article...,속초 체스터톤스 숙박권입니다.\n3매구요~\n원하는 매수 만큼 가능하세요.\n투룸 ...,티켓/교환권,판매중,부천시 여월동,"[속초, 체스터, 톤스, 호텔]",부천시
1,당근마켓,https://www.daangn.com/articles/840996796,2024-10-02 12:52:31.443,수성호텔 vip 티켓 1장,4,0,34000,https://img.kr.gcp-karroter.net/origin/article...,"수성호텔 호텔수성 vip 티켓 (수영장,루프탑,인피니티풀, 온천, 사우나, 이벤트...",티켓/교환권,판매중,북구 매천동,"[수성, 호텔, 티켓, 장]",북구
2,당근마켓,https://www.daangn.com/articles/840996562,2024-10-02 12:52:31.443,"아난티 리조트호텔 가평, 코브, 빌라쥬드, 남해, 제주 숙박권 양도",12,0,540000,https://img.kr.gcp-karroter.net/origin/article...,"남해 빼고 (평일)일-목 체크인 가능, 금토 불가.\n남해 주말권만 금,토 체크인 ...",티켓/교환권,판매중,남구 사직동,"[아난, 티, 리조트, 호텔, 가평, 코브, 빌라, 쥬드, 남해, 제주, 숙박, 양도]",남구
3,당근마켓,https://www.daangn.com/articles/840986215,2024-10-02 12:35:31.443,파타야 힐튼호텔 디럭스룸 10/12~13,21,0,400000,https://img.kr.gcp-karroter.net/origin/article...,연락주세요,티켓/교환권,판매중,수원시 영통구 영통동,"[파타야, 힐튼, 호텔, 디럭스, 룸]",수원시
4,당근마켓,https://www.daangn.com/articles/840984922,2024-10-02 12:33:31.443,글램 호텔 숙박,57,0,60000,https://img.kr.gcp-karroter.net/origin/article...,경희대 글램 호텔 숙박권 판매합니다.\n연락주세요:),티켓/교환권,판매중,종로구 창신제2동,"[글램, 호텔, 숙박]",종로구
5,당근마켓,https://www.daangn.com/articles/840984846,2024-10-02 12:33:31.443,호텔수성 인피니티 수영장 티켓 3장 사봐요!,29,2,100000,https://img.kr.gcp-karroter.net/origin/article...,연락부탁드려용\n답늦을수도있어요!,티켓/교환권,판매중,북구 산격동,"[호텔, 수성, 인피니티, 수영장, 티켓, 장]",북구
6,당근마켓,https://www.daangn.com/articles/840983318,2024-10-02 12:31:31.443,글램 호텔,42,1,60000,https://img.kr.gcp-karroter.net/origin/article...,10월 2일~3일\n오늘 숙박입니다.\n2일 20시 입실\n3일 12시 퇴실\n경희...,티켓/교환권,판매중,광진구 군자동,"[글램, 호텔]",광진구
7,당근마켓,https://www.daangn.com/articles/840983146,2024-10-02 12:30:31.443,주문진 호텔 양도합니다,188,5,80000,https://img.kr.gcp-karroter.net/origin/article...,사정이 생겨서 못가게 되서 양도합니다.,티켓/교환권,판매중,원주시 단구동,"[주문진, 호텔, 니]",원주시
8,당근마켓,https://www.daangn.com/articles/840982152,2024-10-02 12:29:31.443,속초 체스터톤스호텔 금일숙박팔아요,90,8,40000,https://img.kr.gcp-karroter.net/origin/article...,2024년10월2일 금일.숙박권 팝니다\n원룸스탠다드더블룸,티켓/교환권,판매중,동대문구 장안동,"[속초, 체스터, 톤스, 호텔, 금일, 숙박]",동대문구
9,당근마켓,https://www.daangn.com/articles/840979733,2024-10-02 12:25:31.443,하남 씨엘 호텔 대실,51,0,30000,https://img.kr.gcp-karroter.net/origin/article...,하남 씨엘 CL 호텔 오늘 10월 2일 수요일 6시간 대실 (오 후 3시부터 9시까...,티켓/교환권,판매중,강동구 강일동,"[하남, 씨엘, 호텔]",강동구


In [ ]:
from flask import Flask, request, jsonify
from jsonschema import validate, ValidationError

app = Flask(__name__)

# JSON 스키마 정의
schema = {
    "type": "object",
    "properties": {
        "expiration_start_date": {
            "anyOf": [
                {"type": "string", "format": "date"},
                {"type": "null"}
            ],
            "description": "노쇼 상품의 사용 가능 시작 날짜"
        },
        "expiration_end_date": {
            "anyOf": [
                {"type": "string", "format": "date"},
                {"type": "null"}
            ],
            "description": "노쇼 상품의 사용 가능 마지막 날짜"
        },
        "room_type": {
            "anyOf": [
                {"type": "string"},
                {"type": "null"}
            ],
            "description": "객실 타입"
        },
        "head_count": {
            "anyOf": [
                {"type": "integer"},
                {"type": "null"}
            ],
            "description": "사용 인원 수"
        },
        "shipping_fee": {
            "anyOf": [
                {"type": "integer"},
                {"type": "null"}
            ],
            "description": "배송비"
        },
        "transaction_method": {
            "anyOf": [
                {"type": "string"},
                {"type": "null"}
            ],
            "description": "거래 유형 (직거래, 택배거래)"
        },
        "market_price": {
            "anyOf": [
                {"type": "integer"},
                {"type": "null"}
            ],
            "description": "시중 가격"
        },
        "options": {
            "anyOf": [
                {"type": "string"},
                {"type": "null"}
            ],
            "description": "호텔 서비스 옵션 (조식, 수영장 등)"
        },
        "parking": {
            "anyOf": [
                {"type": "boolean"},
                {"type": "null"}
            ],
            "description": "주차 가능 여부"
        },
        "check_in_time": {
            "anyOf": [
                {"type": "string", "pattern": "^([0-1]?[0-9]|2[0-3]):[0-5][0-9]$"},
                {"type": "null"}
            ],
            "description": "체크인 시간 (HH:mm 형식)"
        },
        "check_out_time": {
            "anyOf": [
                {"type": "string", "pattern": "^([0-1]?[0-9]|2[0-3]):[0-5][0-9]$"},
                {"type": "null"}
            ],
            "description": "체크아웃 시간 (HH:mm 형식)"
        },
        "sub_category": {
            "anyOf": [
                {"type": "string"},
                {"type": "null"}
            ],
            "description": "세부 카테고리 (호텔, 리조트, 펜션/콘도 등)"
        }
    },
    "required": [
        "expiration_start_date",
        "expiration_end_date",
        "room_type",
        "head_count",
        "shipping_fee",
        "transaction_method",
        "market_price",
        "options",
        "parking",
        "check_in_time",
        "check_out_time"
    ]
}

# 필드 추출 API
@app.route('/extract_fields', methods=['POST'])
def extract_fields():
    data = request.get_json()

    # 스키마 검증
    try:
        validate(instance=data, schema=schema)
    except ValidationError as e:
        return jsonify({"error": f"Invalid data: {e.message}"}), 400

    # 필드 추출
    extracted_data = {
        "expiration_start_date": data.get("expiration_start_date"),
        "expiration_end_date": data.get("expiration_end_date"),
        "room_type": data.get("room_type"),
        "head_count": data.get("head_count"),
        "shipping_fee": data.get("shipping_fee"),
        "transaction_method": data.get("transaction_method"),
        "market_price": data.get("market_price"),
        "options": data.get("options"),
        "parking": data.get("parking"),
        "check_in_time": data.get("check_in_time"),
        "check_out_time": data.get("check_out_time"),
        "sub_category": data.get("sub_category")
    }

    return jsonify(extracted_data), 200

if __name__ == '__main__':
    app.run(debug=True)


In [11]:
class NoShowProductExtraction(BaseModel):
    expiration_start_date: str = None
    expiration_end_date: str = None
    room_type: str = None
    head_count: int = None
    shipping_fee: int = None
    transaction_method: str = None
    market_price: int = None
    options: str = None
    parking: bool = None
    check_in_time: str = None
    check_out_time: str = None
    sub_category: str = None

# 결과를 저장할 리스트
results = []

# 데이터프레임을 100개씩 처리
for index, row in test_df.iterrows():
    # 데이터 추출에 필요한 내용 작성
    content = f"""
    제목: {row['title']}, 설명: {row['description']}, 업로드 시간: {row['post_time']}
    가격: {row['price']}
    """

    # API 호출
    completion = openai.ChatCompletion.create(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "You are an expert at structured data extraction. You will be given unstructured text from a no-show product description and should convert it into the given structure."},
            {"role": "user", "content": content}
        ]
    )
    
    # 결과 저장
    no_show_product = completion.choices[0].message.content

    # 필드가 누락된 경우 None으로 설정
    no_show_data = NoShowProductExtraction(**{
        "expiration_start_date": no_show_product.get("expiration_start_date", None),
        "expiration_end_date": no_show_product.get("expiration_end_date", None),
        "room_type": no_show_product.get("room_type", None),
        "head_count": no_show_product.get("head_count", None),
        "shipping_fee": no_show_product.get("shipping_fee", None),
        "transaction_method": no_show_product.get("transaction_method", None),
        "market_price": no_show_product.get("market_price", None),
        "options": no_show_product.get("options", None),
        "parking": no_show_product.get("parking", None),
        "check_in_time": no_show_product.get("check_in_time", None),
        "check_out_time": no_show_product.get("check_out_time", None),
        "sub_category": no_show_product.get("sub_category", None)
    })

    # 결과를 리스트에 추가
    results.append(no_show_data)

# 결과를 데이터프레임으로 변환
extracted_df = pd.DataFrame(results)
extracted_df

AttributeError: 'str' object has no attribute 'get'

In [9]:
extracted_df.to_csv('json_test30.csv', index=False, encoding='utf-8-sig')

In [9]:
def extract_fields(description):
    prompt = f"""
    우리는 여러 플랫폼에서 크롤링한 데이터를 통해 호텔과 숙박에 관한 노쇼 상품들을 판매할거야. 
    기본 필드에서 가져올 수 있는 정보들을 제외하고 우리가 필요한 필드가 11개가 있어.
    만약 NaN이라면 그냥 모든 필드를 JSON의 `null`로 표시해줘.
    이 11개의 필드는 
    1. 노쇼 상품의 사용 가능 시작 날짜를 나타내는 expiration_start_date
       설명과 제목, 업로드 시간을 참고해서 시작 날짜를 적어줘. 이때 시작 날짜가 없다면 JSON의 `null`로 표기해줘. 날짜 형식은 20xx-mm-dd로 통일해줘.
    2. 노쇼 상품의 사용 가능 마지막 날짜 및 기한을 나타내는 expiration_end_date
       설명과 제목, 업로드 시간을 참고해서 사용 가능한 마지막 날짜를 적어줘. 이때 마지막 날짜가 없다면 JSON의 `null`로 표기해줘. 날짜 형식은 20xx-mm-dd로 통일해줘.
    3. 객실 타입을 나타내는 room_type
       설명과 제목을 참고해서 적어줘. 문자열 형태이며 관련된 내용이 없다면 JSON의 `null`로 입력해줘.
    4. 사용 인원 수를 나타내는 head_count
       설명과 제목을 참고해서 적어줘. 데이터 타입은 int type이고 성인이나 자녀에 대한 구분 없이 총 인원수만 입력해줘. 인원수에 대한 내용이 없다면 JSON의 `null`로 기입해줘.
    5. 배송비인 shipping_fee 
       설명과 제목을 참고해서 적어줘. 데이터 타입은 int type이고, 배송비가 적혀있지 않다면 0으로 표기해줘 (숫자 형식).
    6. 직거래인지 택배거래인지 알 수 있는 거래 유형인 transaction_method
       설명과 제목을 참고해서 적어줘. 직거래라는 단어가 있으면 '직거래'로, 택배거래라는 단어가 있으면 '택배거래'로 표기해줘.
       두 단어가 모두 없으면 JSON의 `null`로 표기해줘.
    7. 원래 노쇼 상품의 가격인 시중 가격인 market_price 
       설명과 제목을 참고해서 적어줘. 데이터 타입은 int type이고, 시중 가격이 나와 있지 않다면 JSON의 `null`로 표기해줘
       만약 23.3만원 이런식으로 적혀져 있다면 market_price는 233000으로 입력되어야해.
    8. 조식 여부나 사우나 여부, 수영장 여부 등의 호텔에서 사용할 수 있는 서비스 이용 여부를 알 수 있는 옵션인 options
       설명과 제목을 참고해서 적어줘. 서비스 관련 내용이 없다면 JSON의 `null`로 표기해줘.
    9. 주차 여부를 나타내는 parking
       설명과 제목을 참고해서 적어줘. 주차 여부가 있으면 TRUE로, 없으면 FALSE로 표기해줘. 주차 여부 정보가 없다면 JSON의 `null`로 표기해줘.
    10.호텔 체크인 시간을 나타내는 check_in_time
       설명과 제목을 참고해서 적어줘. 체크인 시간을 time 형태로 작성해줘. 만약 체크인 시간이 없으면 JSON의 `null`로 작성해줘.
    11.호텔 체크아웃 시간을 나타내는 check_out_time
       설명과 제목을 참고해서 적어줘. 체크아웃 시간을 time 형태로 작성해줘. 만약 체크아웃 시간이 없으면 JSON의 `null`로 작성해줘.
    12.세부 카테고리를 알 수 있는 sub_category
       설명과 제목을 참고해서 적어줘. 호텔, 노보텔이라는 단어가 있다면 호텔로 작성해줘. 리조트라는 단어가 있다면 리조트로 작성해줘. 
       펜션, 콘도라는 단어가 있다면 펜션/콘도으로 작성해줘.    
       
    우리가 원하는 설명에서 필요한 11개의 필드 정보만 추출해서 json 형태로 작성해줘:
    예시: 

        "expiration_start_date": "2023-10-01",
        "expiration_end_date": "2023-10-05",
        "room_type": "더블룸",
        "head_count": 2,
        "shipping_fee": 0,
        "transaction_method": "직거래",
        "market_price": 100000,
        "options": "조식 가능, 수영장 사용 가능",
        "parking": True,
        "check_in_time": 10:00,
        "check_out_time": 15:00
    
    
    제목 : {title}, 설명: {description}, 업로드 시간 : {post_time}

    필드:
    - expiration_start_date:
    - expiration_end_date:
    - room_type:
    - head_count:
    - shipping_fee:
    - transaction_method:
    - market_price:
    - options:
    - parking:
    - check_in_time:
    - check_out_time:
    """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
        max_tokens=1000
    )
    

    result = response['choices'][0]['message']['content'].strip()
    
    print("API 반환 결과:", result)
    
# json으로 시작되는 쓸데없는 부분을 제거 / 끝부분에 '들어가는 부분 제거 코드
    if result.startswith("```json"):
        result = result.replace("```json", "").strip()
    if result.endswith("```"):
        result = result.replace("```", "").strip()
        
   # 'null'을 실제 JSON의 null로 변환
    result = result.replace('"null"', 'null')

    try:
        return json.loads(result)
    except json.JSONDecodeError:
        print("유효하지 않은 JSON 형식입니다.")
        return None

from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class NoShowProductExtraction(BaseModel):
    expiration_start_date: str
    expiration_end_date: str
    room_type: str
    head_count: int
    shipping_fee: int
    transaction_method: str
    market_price: int
    options: str
    parking: bool
    check_in_time: str
    check_out_time: str
    sub_category: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are an expert at structured data extraction. You will be given unstructured text from a no-show product description and should convert it into the given structure."},
        {"role": "user", "content": """
        우리는 여러 플랫폼에서 크롤링한 데이터를 통해 호텔과 숙박에 관한 노쇼 상품들을 판매할거야.
        제목 : {title}, 설명: {description}, 업로드 시간 : {post_time}
        """}
    ],
    response_format=NoShowProductExtraction,
)

no_show_product = completion.choices[0].message.parsed

In [10]:
df_sample=df['description'].sample(30)

In [11]:
json_to_df_sample= df_sample.apply(extract_fields)
# 데이터프레임으로 만드는 코드
fields_df_sample = pd.json_normalize(json_to_df_sample)

# fields_df와 df를 열(column) 기준으로 병합 (옆으로 붙이기)
merged_df_sample = pd.concat([df_sample, fields_df_sample], axis=1)

# 병합된 데이터 확인
print(merged_df_sample)

API 반환 결과: ```json
{
    "expiration_date": "null",
    "room_type": "null",
    "head_count": "null",
    "shipping_fee": 0,
    "transaction_method": "직거래",
    "market_price": "null",
    "options": "null",
    "parking": "null",
    "check_in_time": "null",
    "check_out_time": "null"
}
```
API 반환 결과: {
    "expiration_date": "2023-10-11 ~ 2023-10-12",
    "room_type": null,
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": "직거래",
    "market_price": 120000,
    "options": null,
    "parking": true,
    "check_in_time": null,
    "check_out_time": null
}
API 반환 결과: ```json
{
    "expiration_date": "2024-12-28",
    "room_type": "슈페리어 스위트룸",
    "head_count": 0,
    "shipping_fee": 0,
    "transaction_method": "택배거래",
    "market_price": 341960,
    "options": "피트니스, 수영장, 사우나 무료",
    "parking": null,
    "check_in_time": null,
    "check_out_time": null
}
```
API 반환 결과: ```json
{
    "expiration_date": "2023-12-31",
    "room_type": null,
    "head_count": n

In [16]:
merged_df_sample

,description,expiration_date,room_type,head_count,shipping_fee,transaction_method,market_price,options,parking,check_in_time,check_out_time
450,오늘 가는날인데 애가 아파서 올립니다.\n010-5270-9874 연락주세요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
534,1박이구용\n금-토입니다 ㅠ 10월11-12일입니다\n\n12만원주고 예약했는데 9...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,글 안 읽고 메세지 보내면 답장 안드립니다. 바우처 예약은 일별 예약 가능한 객실 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,제주 해비치 호텔 수페리어룸 숙박권 판매합니다.\n​\n호텔 이용 시 섬모라 조식 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,새상품\n\n모텔 호텔 카드\n물려잇으면 모텔이나 호텔방\n전원나가지 않아 전기공급...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,"- 이그제티브 트윈 바다전망 고층 \n20층피트니스포함\n\n- 싱글침대1, 더블침...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291,"** 그랜드테이블 조식 부페권 **\n\n대인 정상가 49,000원 >>> 대인 할...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537,롯데호텔 서울 숙박권 판매합니다.\n\n이그제큐티브 타워\n디럭스룸2인1박+라세느 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,대구 인터불고호텔 이번주말 숙박 양도해줄분 정가양도 합니다,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,오늘 코엑스 글래드호텔 숙박권 팝니다.\n제가 오늘 갈 예정이었는데 급하게 취소되서...,2023-12-31,스위트룸,NaN,0.0,택배거래,450000.0,None,True,15:00,11:00


In [13]:
merged_df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 450 to 28
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   description         30 non-null     object 
 1   expiration_date     13 non-null     object 
 2   room_type           14 non-null     object 
 3   head_count          10 non-null     float64
 4   shipping_fee        30 non-null     float64
 5   transaction_method  15 non-null     object 
 6   market_price        13 non-null     float64
 7   options             15 non-null     object 
 8   parking             9 non-null      object 
 9   check_in_time       3 non-null      object 
 10  check_out_time      2 non-null      object 
dtypes: float64(3), object(8)
memory usage: 5.4+ KB


In [15]:
json_to_df= df['description'].apply(extract_fields)
# 데이터프레임으로 만드는 코드
fields_df = pd.json_normalize(json_to_df)

# fields_df와 df를 열(column) 기준으로 병합 (옆으로 붙이기)
merged_df = pd.concat([df, fields_df], axis=1)

# 병합된 데이터 확인
print(merged_df)

merged_df.to_csv('prompt_df.csv', index=False, encoding='utf-8-sig')

API 반환 결과: ```json
{
    "expiration_date": "null",
    "room_type": "투룸 슈페리어 더블",
    "head_count": 6,
    "shipping_fee": 0,
    "transaction_method": "직거래",
    "market_price": null,
    "options": "수영장 사용 가능",
    "parking": true,
    "check_in_time": null,
    "check_out_time": null
}
```
API 반환 결과: ```json
{
    "expiration_date": "null",
    "room_type": null,
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": null,
    "market_price": null,
    "options": "수영장, 루프탑, 인피니티풀, 온천, 사우나, 이벤트탕",
    "parking": null,
    "check_in_time": null,
    "check_out_time": null
}
```
API 반환 결과: ```json
{
    "expiration_date": "2023-01-02",
    "room_type": null,
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": "직거래",
    "market_price": 560000,
    "options": "식사크레딧 없음",
    "parking": true,
    "check_in_time": null,
    "check_out_time": null
}
```
API 반환 결과: ```json
{
    "expiration_date": null,
    "room_type": null,
    "head_count": null,
 

KeyboardInterrupt: 

In [42]:
merged_df.to_csv('prompt_df.csv', index=False, encoding='utf-8-sig')

In [17]:
fields_df.info()

NameError: name 'fields_df' is not defined

# 여기서부터는 무시

In [132]:
df.head(10)

,platform,original_link,post_time,title,view_count,like_count,price,images,description,category,status,seller_location,title_nouns,seller_location_1
0,당근마켓,https://www.daangn.com/articles/832373103,2024-10-02 12:53:31.443,속초 체스터톤스 호텔,310,7,170000,https://img.kr.gcp-karroter.net/origin/article...,속초 체스터톤스 숙박권입니다.\n3매구요~\n원하는 매수 만큼 가능하세요.\n투룸 ...,티켓/교환권,판매중,부천시 여월동,"[속초, 체스터, 톤스, 호텔]",부천시
1,당근마켓,https://www.daangn.com/articles/840996796,2024-10-02 12:52:31.443,수성호텔 vip 티켓 1장,4,0,34000,https://img.kr.gcp-karroter.net/origin/article...,"수성호텔 호텔수성 vip 티켓 (수영장,루프탑,인피니티풀, 온천, 사우나, 이벤트...",티켓/교환권,판매중,북구 매천동,"[수성, 호텔, 티켓, 장]",북구
2,당근마켓,https://www.daangn.com/articles/840996562,2024-10-02 12:52:31.443,"아난티 리조트호텔 가평, 코브, 빌라쥬드, 남해, 제주 숙박권 양도",12,0,540000,https://img.kr.gcp-karroter.net/origin/article...,"남해 빼고 (평일)일-목 체크인 가능, 금토 불가.\n남해 주말권만 금,토 체크인 ...",티켓/교환권,판매중,남구 사직동,"[아난, 티, 리조트, 호텔, 가평, 코브, 빌라, 쥬드, 남해, 제주, 숙박, 양도]",남구
3,당근마켓,https://www.daangn.com/articles/840986215,2024-10-02 12:35:31.443,파타야 힐튼호텔 디럭스룸 10/12~13,21,0,400000,https://img.kr.gcp-karroter.net/origin/article...,연락주세요,티켓/교환권,판매중,수원시 영통구 영통동,"[파타야, 힐튼, 호텔, 디럭스, 룸]",수원시
4,당근마켓,https://www.daangn.com/articles/840984922,2024-10-02 12:33:31.443,글램 호텔 숙박,57,0,60000,https://img.kr.gcp-karroter.net/origin/article...,경희대 글램 호텔 숙박권 판매합니다.\n연락주세요:),티켓/교환권,판매중,종로구 창신제2동,"[글램, 호텔, 숙박]",종로구
5,당근마켓,https://www.daangn.com/articles/840984846,2024-10-02 12:33:31.443,호텔수성 인피니티 수영장 티켓 3장 사봐요!,29,2,100000,https://img.kr.gcp-karroter.net/origin/article...,연락부탁드려용\n답늦을수도있어요!,티켓/교환권,판매중,북구 산격동,"[호텔, 수성, 인피니티, 수영장, 티켓, 장]",북구
6,당근마켓,https://www.daangn.com/articles/840983318,2024-10-02 12:31:31.443,글램 호텔,42,1,60000,https://img.kr.gcp-karroter.net/origin/article...,10월 2일~3일\n오늘 숙박입니다.\n2일 20시 입실\n3일 12시 퇴실\n경희...,티켓/교환권,판매중,광진구 군자동,"[글램, 호텔]",광진구
7,당근마켓,https://www.daangn.com/articles/840983146,2024-10-02 12:30:31.443,주문진 호텔 양도합니다,188,5,80000,https://img.kr.gcp-karroter.net/origin/article...,사정이 생겨서 못가게 되서 양도합니다.,티켓/교환권,판매중,원주시 단구동,"[주문진, 호텔, 니]",원주시
8,당근마켓,https://www.daangn.com/articles/840982152,2024-10-02 12:29:31.443,속초 체스터톤스호텔 금일숙박팔아요,90,8,40000,https://img.kr.gcp-karroter.net/origin/article...,2024년10월2일 금일.숙박권 팝니다\n원룸스탠다드더블룸,티켓/교환권,판매중,동대문구 장안동,"[속초, 체스터, 톤스, 호텔, 금일, 숙박]",동대문구
9,당근마켓,https://www.daangn.com/articles/840979733,2024-10-02 12:25:31.443,하남 씨엘 호텔 대실,51,0,30000,https://img.kr.gcp-karroter.net/origin/article...,하남 씨엘 CL 호텔 오늘 10월 2일 수요일 6시간 대실 (오 후 3시부터 9시까...,티켓/교환권,판매중,강동구 강일동,"[하남, 씨엘, 호텔]",강동구
